In [188]:
from pytrack.data_mgmt import * 
from pytrack.game_moments import * 
from pytrack.positions_analysis import * 
from pytrack.pitch_creation import * 
from pytrack.positions_analysis import * 
from pytrack.build_up import * 
from tracking_config import *

#from moviepy.editor import VideoFileClip, vfx
import glob
from google.cloud import bigquery
from pandas.io import gbq
import pandas_gbq
import plotly_express as px
import matplotlib.pyplot as plt
from forel_clustering_master import *
import os
import sys
import bisect
from bisect import bisect
from sklearn.cluster import KMeans
import warnings
from collections import Counter
import logging
import glob

go = 2
print('Let it go')

In [189]:
os.getcwd()

'/Users/vanni.DiFebo/env'

In [190]:
def filter_find_game_id_f7(homeTeam_str, awayTeam_str, competitionCountry, competitionName__, competitionSearchFlag, matchday, seasonId, month_, avoid_under, avoid_women_teams, avoid_women_under):

    if competitionSearchFlag == 'exact':
        base_query = 'SELECT competitionCountry, optaCompName, game_id, stat_matchday, matchDate, homeTeamName, awayTeamName FROM `the-fa-analysis-and-insight.source_external_opta.opta_f7_detailed_table` WHERE (optaCompName = "'+str(competitionName__)+'" AND EXTRACT(YEAR FROM matchDate) = '+str(seasonId)+')'
        
    else:
        base_query = 'SELECT competitionCountry, optaCompName, game_id, stat_matchday, matchDate, homeTeamName, awayTeamName FROM `the-fa-analysis-and-insight.source_external_opta.opta_f7_detailed_table` WHERE (optaCompName LIKE "%'+str(competitionName__)+'%" AND EXTRACT(YEAR FROM matchDate) = '+str(seasonId)+')'

    if month_ != '':
        base_query = base_query[:-1] + ' AND EXTRACT(MONTH FROM matchDate) = '+str(month_)+')'
        
    print(competitionName)
    # Add conditions based on the provided variables
    conditions = []
    
    if homeTeam_str != '':
        conditions.append(f"homeTeamName LIKE '%{homeTeam_str}%'")
    if awayTeam_str != '':
        conditions.append(f"awayTeamName LIKE '%{awayTeam_str}%'")
    if competitionCountry != '':
        conditions.append(f"competitionCountry LIKE '%{competitionCountry}%'")
    if matchday != '':
        conditions.append(f"stat_matchday = {matchday}")
    if avoid_women_teams == 'Yes':
            conditions.append(f"homeTeamName NOT LIKE '%Women%'")
            conditions.append(f"awayTeamName NOT LIKE '%Women%'")
        
    if avoid_women_under == 'Yes':
        for under_team in ['U23','U21','U20','U19','U18','U17','U16','U15']:
            conditions.append(f"homeTeamName NOT LIKE '%Women {under_team}%'")
            conditions.append(f"awayTeamName NOT LIKE '%Women {under_team}%'")
   
    if avoid_under == 'Yes':
        for under_team in ['U23','U21','U20','U19','U18','U17','U16','U15']:
            conditions.append(f"homeTeamName NOT LIKE '%{under_team}%'")
            conditions.append(f"awayTeamName NOT LIKE '%{under_team}%'")
    elif avoid_under == 'Only':
        for under_team in ['U23','U20','U18','U16','U15']:
            conditions.append(f"homeTeamName NOT LIKE '%{under_team}%'")
            conditions.append(f"awayTeamName NOT LIKE '%{under_team}%'")
    
    
    # Combine the base query with the conditions
    if conditions:
        base_query += " AND " + " AND ".join(conditions)
    
    df_fixtures = pandas_gbq.read_gbq(base_query, project_id = 'the-fa-analysis-and-insight')
    
    df_fixtures_r = df_fixtures.rename(columns = {'optaCompName':'competitionName', 'game_id':'matchId', 'stat_matchday':'matchday', 'matchDate':'matchDateTime', 'homeTeamName':'homeTeam', 'awayTeamName':'awayTeam'})
    
    df_fixtures_rs = df_fixtures_r[['competitionCountry', 'competitionName', 'matchId', 'matchday', 'matchDateTime', 'homeTeam', 'awayTeam']].drop_duplicates()
    
    return df_fixtures_rs

    
if go == 1:
    team_name = 'England U21'
    replaceOrAppend = 'replace'

elif go == 2:
    team_name = 'England U18'
    replaceOrAppend = 'append'

else:
    pass

if team_name == 'England U21' :
    keep_competitions =  ['UEFA Under 21 Championship', 'UEFA Under 21 Championship Qualifiers', 'Under 21 Internationals']
else:
    keep_competitions =  ['FIFA U17 World Cup','Under 18 Internationals'] 

use_fixed = ''
reset_ = 'Yes'
avoid_under = 'No'
avoid_women_teams = 'Yes'
avoid_women_under = 'Yes'

data_version_label = '1.3.19'
test_combos_to_compute = []

find_game_id_filt = pd.DataFrame()

starting_month = 1
ending_month = 12
starting_year = 2021
ending_year = 2025

for yr_ in list(np.arange(starting_year,ending_year+1,1)):

    for comp_ in keep_competitions:
        
        test_combos_to_compute.append(['','','',comp_,'','',yr_,''])
    
    
for combo in test_combos_to_compute:
    
        [homeTeam_str,awayTeam_str,competitionCountry,competitionName,competitionSearchFlag,matchday,seasonId, month_] = combo
        
        app_find_game_id_filt = filter_find_game_id_f7(homeTeam_str, awayTeam_str, competitionCountry, competitionName, competitionSearchFlag, matchday, seasonId, month_, avoid_under, avoid_women_teams, avoid_women_under)
            
        # Reset indices of both DataFrames
        find_game_id_filt.reset_index(drop=True, inplace=True)
        app_find_game_id_filt.reset_index(drop=True, inplace=True)
        #print('')
        #print('Appended '+str(combo))
        # Concatenate DataFrames while ignoring the index
        find_game_id_filt = pd.concat([find_game_id_filt, app_find_game_id_filt], ignore_index=True)
        find_game_id_filt = find_game_id_filt.drop_duplicates()

        if len(app_find_game_id_filt) > 0:
            print(combo)
            print('Appended: '+str(len(app_find_game_id_filt))+' games')
            print(' ')
        
print(find_game_id_filt.sort_values(by = ['matchId'], ascending = False))


FIFA U17 World Cup
Downloading: |                                                                 |
Under 18 Internationals
Downloading: 100%|█████████████████████████████████████████████████████████████|
['', '', '', 'Under 18 Internationals', '', '', np.int64(2021), '']
Appended: 13 games
 
FIFA U17 World Cup
Downloading: |                                                                 |
Under 18 Internationals
Downloading: 100%|█████████████████████████████████████████████████████████████|
['', '', '', 'Under 18 Internationals', '', '', np.int64(2022), '']
Appended: 11 games
 
FIFA U17 World Cup
Downloading: 100%|█████████████████████████████████████████████████████████████|
['', '', '', 'FIFA U17 World Cup', '', '', np.int64(2023), '']
Appended: 52 games
 
Under 18 Internationals
Downloading: 100%|█████████████████████████████████████████████████████████████|
['', '', '', 'Under 18 Internationals', '', '', np.int64(2023), '']
Appended: 27 games
 
FIFA U17 World Cup
Downloading: | 

In [191]:
find_game_id_filt[['matchId','competitionName']].drop_duplicates().groupby(by = ['competitionName']).count()

,matchId
competitionName,
FIFA U17 World Cup,156
Under 18 Internationals,108


In [192]:
find_game_id_filt[find_game_id_filt['competitionName'] == 'UEFA Under 21 Championship'].sort_values(by = ['matchId'])

,competitionCountry,competitionName,matchId,matchday,matchDateTime,homeTeam,awayTeam


In [193]:
games_string_joined = ', '.join(map(str, list(find_game_id_filt.sort_values(by = ['matchDateTime'])['matchId'].unique())))

#games_string_joined = '2180288, 2180289, 2180294, 2180295, 2189480, 2180307, 2180306, 2180300, 2180301, 2180290, 2180291, 2180296, 2180297, 2180309, 2180303, 2180308, 2180302, 2189481, 2180299, 2180298, 2180293, 2180292, 2180304, 2180305, 2180310'

games_string_joined

'2200777, 2245009, 2245368, 2256819, 2256820, 2254562, 2258557, 2258809, 2258558, 2258559, 2258810, 2258839, 2258842, 2283809, 2283810, 2286817, 2286818, 2286819, 2332091, 2332217, 2332280, 2410273, 2410274, 2410765, 2359950, 2359311, 2360022, 2359312, 2360129, 2359313, 2365667, 2365746, 2365747, 2365745, 2365957, 2409939, 2409197, 2409297, 2409353, 2409355, 2411357, 2411546, 2411769, 2411548, 2411954, 2411547, 2412319, 2413033, 2413034, 2409810, 2409812, 2409811, 2409809, 2409813, 2409816, 2409815, 2409814, 2409819, 2409817, 2409820, 2409818, 2409821, 2409822, 2409824, 2409823, 2409827, 2409826, 2409828, 2409825, 2409829, 2409831, 2409832, 2409830, 2413268, 2409836, 2409835, 2409833, 2409834, 2409837, 2409839, 2409838, 2409840, 2413269, 2409841, 2409844, 2409842, 2409843, 2409846, 2409845, 2409849, 2409848, 2409852, 2409851, 2409850, 2409847, 2409854, 2409853, 2409855, 2409856, 2409858, 2409857, 2409859, 2409860, 2431912, 2438006, 2431913, 2438169, 2431914, 2439769, 2441364, 2441418, 

In [194]:
find_game_id_filt[(find_game_id_filt['homeTeam'].str.contains('England'))]

,competitionCountry,competitionName,matchId,matchday,matchDateTime,homeTeam,awayTeam
0,World,Under 18 Internationals,2256820,1,2021-10-11,England U18,Russia U18
9,World,Under 18 Internationals,2256819,1,2021-10-07,England U18,Norway U18
17,World,Under 18 Internationals,2283809,1,2022-03-25,England U18,Sweden U18
18,World,Under 18 Internationals,2283810,1,2022-03-28,England U18,Denmark U18
19,World,Under 18 Internationals,2286817,1,2022-06-07,England U18,Austria U18
20,World,Under 18 Internationals,2332091,1,2022-09-21,England U18,Netherlands U18
21,World,Under 18 Internationals,2332217,1,2022-09-23,England U18,Faroe Islands U18
22,World,Under 18 Internationals,2286818,1,2022-06-10,England U18,Wales U18
23,World,Under 18 Internationals,2286819,1,2022-06-13,England U18,Croatia U18
24,None,FIFA U17 World Cup,2409837,3,2023-11-17,England U17,Brazil U17


In [195]:
sql_phases = 'SELECT * FROM `the-fa-performance-insights.phases.phase_windows_new` WHERE game_id IN ('+games_string_joined+')'
joined_phases_dataset = pandas_gbq.read_gbq(sql_phases, project_id = 'the-fa-performance-insights')
pre_joined_phases_dataset = joined_phases_dataset.drop_duplicates()
pre_joined_phases_dataset['time_diff'] = pre_joined_phases_dataset['end_time'] - pre_joined_phases_dataset['start_time']

pre_joined_phases_dataset.head()

Downloading: 100%|█████████████████████████████████████████████████████████████|


,game_id,team_id,phase,label,half,set_piece_type,gk_label,start_time,end_time,time_diff
0,2411546,9459,BUILD,BUILD 1,1,None,GK,27.835,51.947,24.112
1,2411547,9459,BUILD,BUILD 1,1,None,GK,250.544,258.560,8.016
2,2411548,9459,BUILD,BUILD 1,1,None,None,26.174,37.743,11.569
3,2411546,9459,BUILD,BUILD 10,1,ATTACKING FREE KICK,GOAL KICK,928.757,928.757,0.000
4,2411548,9459,BUILD,BUILD 10,1,None,GK,1200.850,1221.399,20.549


In [196]:
pre_joined_phases_dataset[pre_joined_phases_dataset['game_id'] == 2180308]

,game_id,team_id,phase,label,half,set_piece_type,gk_label,start_time,end_time,time_diff


In [197]:
len(pre_joined_phases_dataset.game_id.unique())

209

In [198]:
sql_events = 'SELECT * FROM `the-fa-analysis-and-insight.source_external_opta.opta_true_sequence_with_xG` WHERE game_id IN ('+str(games_string_joined)+')'
pre_joined_events_dataset = pandas_gbq.read_gbq(sql_events, project_id = 'the-fa-analysis-and-insight')

pre_joined_events_dataset.head()

Downloading: 100%|█████████████████████████████████████████████████████████████|


,id,game_id,game_away_team_id,game_away_team_name,game_competition_id,game_competition_name,game_game_date,game_home_team_id,game_home_team_name,game_matchday,...,assisted_xG,assisted_opta_xG,assisted_Big_Chance,shot_state,shot_state_detail,shot_distance,shot_type,event_leading_to_shot,seq_xG,pos_xG
0,2615642137,2409809,9138,Morocco U17,670,FIFA U17 World Cup,2023-11-10 09:00:00+00:00,9139,Panama U17,1,...,NaN,NaN,<NA>,None,None,NaN,None,None,0.0,0.0
1,2615642145,2409809,9138,Morocco U17,670,FIFA U17 World Cup,2023-11-10 09:00:00+00:00,9139,Panama U17,1,...,NaN,NaN,<NA>,None,None,NaN,None,None,0.0,0.0
2,2615642145,2409809,9138,Morocco U17,670,FIFA U17 World Cup,2023-11-10 09:00:00+00:00,9139,Panama U17,1,...,NaN,NaN,<NA>,None,None,NaN,None,None,0.0,0.0
3,2615642179,2409809,9138,Morocco U17,670,FIFA U17 World Cup,2023-11-10 09:00:00+00:00,9139,Panama U17,1,...,NaN,NaN,<NA>,None,None,NaN,None,None,0.0,0.0
4,2615642179,2409809,9138,Morocco U17,670,FIFA U17 World Cup,2023-11-10 09:00:00+00:00,9139,Panama U17,1,...,NaN,NaN,<NA>,None,None,NaN,None,None,0.0,0.0


In [199]:
len(pre_joined_events_dataset.game_id.unique())

210

In [200]:
pre_joined_events_dataset[pre_joined_events_dataset['game_home_team_name'] == 'Spain U21'][['game_home_team_name','game_away_team_name','game_id']].drop_duplicates()

,game_home_team_name,game_away_team_name,game_id


In [201]:
'''
joined_events_dataset = pd.DataFrame()
joined_phases_dataset = pd.DataFrame()

team_name = 'U17_WC_DAY1'
avoid_games = []

# Define the new directory path
folder_path = '/Users/vanni.DiFebo/env/Phase Data '+team_name

list_of_files = os.listdir(folder_path)

events_files = list(filter(lambda d : 'events' in d, list_of_files))
phases_files = list(filter(lambda d : 'phases' in d, list_of_files))

for game_to_avoid in avoid_games:
    events_files = list(filter(lambda row: str(game_to_avoid) not in row, events_files))
    phases_files = list(filter(lambda row: str(game_to_avoid) not in row, phases_files))

# Loop through files in the folder
for filename_ev in events_files:
    file_path_ev = pd.read_csv(folder_path+'/'+filename_ev)
    file_path_ev = file_path_ev.drop(columns = {'Unnamed: 0'})
    joined_events_dataset = pd.concat([joined_events_dataset, file_path_ev], ignore_index = True)
    
for filename_ph in phases_files:
    file_path_ph = pd.read_csv(folder_path+'/'+filename_ph)
    file_path_ph = file_path_ph.drop(columns = {'Unnamed: 0'})
    joined_phases_dataset = pd.concat([joined_phases_dataset, file_path_ph], ignore_index = True)
'''

"\njoined_events_dataset = pd.DataFrame()\njoined_phases_dataset = pd.DataFrame()\n\nteam_name = 'U17_WC_DAY1'\navoid_games = []\n\n# Define the new directory path\nfolder_path = '/Users/vanni.DiFebo/env/Phase Data '+team_name\n\nlist_of_files = os.listdir(folder_path)\n\nevents_files = list(filter(lambda d : 'events' in d, list_of_files))\nphases_files = list(filter(lambda d : 'phases' in d, list_of_files))\n\nfor game_to_avoid in avoid_games:\n    events_files = list(filter(lambda row: str(game_to_avoid) not in row, events_files))\n    phases_files = list(filter(lambda row: str(game_to_avoid) not in row, phases_files))\n\n# Loop through files in the folder\nfor filename_ev in events_files:\n    file_path_ev = pd.read_csv(folder_path+'/'+filename_ev)\n    file_path_ev = file_path_ev.drop(columns = {'Unnamed: 0'})\n    joined_events_dataset = pd.concat([joined_events_dataset, file_path_ev], ignore_index = True)\n    \nfor filename_ph in phases_files:\n    file_path_ph = pd.read_csv(fol

In [202]:
#joined_events_dataset.head()

In [203]:
joined_phases_dataset[(joined_phases_dataset['game_id'] == 2180308) & (joined_phases_dataset['start_time'] > 999)].sort_values(by = ['start_time']).head()

,game_id,team_id,phase,label,half,set_piece_type,gk_label,start_time,end_time


In [204]:
'''
event_type_name
set_piece_phase
set_piece_type
home_team_phases
home_team_phases_label
away_team_phases
away_team_phases_label
'''

def describe_opta_event_types(ev_type_id):

        dict_mapping_types =  {
            1: "Pass", 2: "Offside Pass", 3: "Take On", 4: "Foul",
            5: "Out", 6: "Corner Awarded", 7: "Tackle", 8: "Interception",
            10: "Save", 11: "Claim", 12: "Clearance", 13: "Miss",
            14: "Post", 15: "Attempt Saved", 16: "Goal", 17: "Card",
            18: "Player Off", 19: "Player On", 20: "Player Retired",
            21: "Player Returns", 22: "Player Becomes Goalkeeper",
            23: "Goalkeeper Becomes Player", 24: "Condition Change",
            25: "Official Change", 27: "Start Delay", 28: "End Delay",
            30: "End", 32: "Start", 34: "Team Set Up",
            35: "Player Changed Position", 36: "Player Changed Jersey Number",
            37: "Collection End", 38: "Temp Goal", 39: "Temp Attempt",
            40: "Formation Change", 41: "Punch", 42: "Good Skill",
            43: "Deleted Event", 44: "Aerial", 45: "Challenge",
            49: "Ball Recovery", 50: "Dispossessed", 51: "Error",
            52: "Keeper Pick-up", 53: "Cross Not Claimed", 54: "Smother",
            55: "Offside Provoked", 56: "Shield Ball Opp", 57: "Foul Throw-in",
            58: "Penalty Faced", 59: "Keeper Sweeper", 60: "Chance Missed",
            61: "Ball Touch", 63: "Temp Save", 64: "Resume",
            65: "Contentious Referee Decision", 67: "50/50",
            68: "Referee Drop Ball", 70: "Injury Time Announcement",
            71: "Coach Setup", 74: "Blocked Pass", 75: "Delayed Start",
            76: "Early End", 79: "Coverage Interruption", 80: "Drop of Ball",
            81: "Obstacle", 82: "Control", 83: "Attempted Tackle",
            84: "Deleted After Review", 124: "Goal Kick"
        }

        return dict_mapping_types[ev_type_id]

def find_phase_from_table(row, team_id, phases_table):

    game_id = row['game_id']
    gameClock = row['gameClock']
    event_period_id = row['event_period_id']
    home_team_id = row['game_home_team_id']
    away_team_id = row['game_away_team_id']

    if event_period_id == 2:
        gameClock_new = gameClock-2700
    else:
        gameClock_new = gameClock
        
    filt_phases = phases_table[
        (phases_table['game_id'] == int(game_id)) &
        (phases_table['half'] == int(event_period_id)) &
        (phases_table['team_id'] == int(team_id)) &
        (phases_table['start_time'] <= float(gameClock_new)) &
        (phases_table['end_time'] >= float(gameClock_new))
    ]
    
    try:
        return pd.Series(filt_phases[['phase','label']].iloc[0].values.tolist())
    except:
        return pd.Series([None, None])

def find_set_piece_phase_from_table(row, team_id, phases_table):

    game_id = row['game_id']
    gameClock = row['gameClock']
    event_period_id = row['event_period_id']
    home_team_id = row['game_home_team_id']
    away_team_id = row['game_away_team_id']

    if event_period_id == 2:
        gameClock_new = gameClock-2700
    else:
        gameClock_new = gameClock
        
    filt_phases = phases_table[
        (phases_table['game_id'] == int(game_id)) &
        (phases_table['half'] == int(event_period_id)) &
        (phases_table['team_id'] == int(team_id)) &
        (phases_table['start_time'] <= float(gameClock_new)) &
        (phases_table['end_time'] >= float(gameClock_new))
    ]
    
    try:
        return pd.Series(filt_phases[['set_piece_type','label']].iloc[0].values.tolist())
    except:
        return pd.Series([None, None])

pre_joined_events_dataset_copy = pre_joined_events_dataset.copy()

pre_joined_events_dataset_copy['event_type_name'] = pre_joined_events_dataset_copy.apply(lambda row: describe_opta_event_types(row.event_type_id), axis = 1)
pre_joined_events_dataset_copy[['home_team_phases','home_team_phases_label']] = pre_joined_events_dataset_copy.apply(lambda row: find_phase_from_table(row, row.game_home_team_id, joined_phases_dataset), axis = 1)
pre_joined_events_dataset_copy[['away_team_phases','away_team_phases_label']] = pre_joined_events_dataset_copy.apply(lambda row: find_phase_from_table(row, row.game_away_team_id, joined_phases_dataset), axis = 1)


In [205]:
display_all(pre_joined_events_dataset_copy[(pre_joined_events_dataset_copy['game_id'].isin([2180299, 2180313, 2180296])) & (pre_joined_events_dataset_copy['gameClock'] < 50)].groupby(by = ['gameClock','event_period_id','game_home_team_id','game_id','home_team_phases']).count())


,,,,,id,game_away_team_id,game_away_team_name,game_competition_id,game_competition_name,game_game_date,game_home_team_name,game_matchday,game_season_id,game_season_name,event_id,event_event_id,event_type_id,event_period_id_text,event_version,event_min,event_sec,event_player_id,event_outcome,event_timestamp,qualifier,lastModified,sequence_id,f24_event_type,event_team_id,event_type_order,event_assist,event_keypass,prev_possession_id,prev_event_team_id,prev_event_period_id,event_time,possession_id,event_team_no,event_sequence_no,Period,possession_id2,start_x,start_y,crosses,head_pass,set_play,successful_through_balls,successful_crosses,successful_passes_to_final_3rd,Player,primary_position,receiver,event_receiver_id,carry_x,carry_y,shot_y,blocked_x,blocked_y,through_balls_received,crosses_received,event_possession_id2_no,prev_end_x,prev_end_y,percent_of_carry_distance_remaining_to_goal,percent_of_pass_distance_remaining_to_goal,vertical_pass_fwd_distance,vertical_pass_distance,horizontal_pass_distance,vertical_carry_fwd_distance,vertical_carry_distance,horizontal_carry_distance,pass_distance,pass_start_pitch_zone,pass_end_pitch_zone,pass_angle,seq_shot,pos_shot,pos_passes,pos_successful_passes,event_x,event_y,end_x,end_y,EVENT,path,xG,Opta_xG,Opta_xGOT,assisted_xG,assisted_opta_xG,assisted_Big_Chance,shot_state,shot_state_detail,shot_distance,shot_type,event_leading_to_shot,seq_xG,pos_xG,event_type_name,home_team_phases_label,away_team_phases,away_team_phases_label
gameClock,event_period_id,game_home_team_id,game_id,home_team_phases,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,


In [206]:
display_all(pre_joined_events_dataset_copy[pre_joined_events_dataset_copy['game_id'] == 2180308].head())

,id,game_id,game_away_team_id,game_away_team_name,game_competition_id,game_competition_name,game_game_date,game_home_team_id,game_home_team_name,game_matchday,game_season_id,game_season_name,event_id,event_event_id,event_type_id,event_period_id,event_period_id_text,event_version,event_min,event_sec,event_player_id,event_outcome,event_timestamp,qualifier,lastModified,sequence_id,f24_event_type,event_team_id,event_type_order,event_assist,event_keypass,prev_possession_id,prev_event_team_id,prev_event_period_id,event_time,possession_id,event_team_no,event_sequence_no,Period,gameClock,possession_id2,start_x,start_y,crosses,head_pass,set_play,successful_through_balls,successful_crosses,successful_passes_to_final_3rd,Player,primary_position,receiver,event_receiver_id,carry_x,carry_y,shot_y,blocked_x,blocked_y,through_balls_received,crosses_received,event_possession_id2_no,prev_end_x,prev_end_y,percent_of_carry_distance_remaining_to_goal,percent_of_pass_distance_remaining_to_goal,vertical_pass_fwd_distance,vertical_pass_distance,horizontal_pass_distance,vertical_carry_fwd_distance,vertical_carry_distance,horizontal_carry_distance,pass_distance,pass_start_pitch_zone,pass_end_pitch_zone,pass_angle,seq_shot,pos_shot,pos_passes,pos_successful_passes,event_x,event_y,end_x,end_y,EVENT,path,xG,Opta_xG,Opta_xGOT,assisted_xG,assisted_opta_xG,assisted_Big_Chance,shot_state,shot_state_detail,shot_distance,shot_type,event_leading_to_shot,seq_xG,pos_xG,event_type_name,home_team_phases,home_team_phases_label,away_team_phases,away_team_phases_label


In [207]:
pre_joined_events_dataset_copy[pre_joined_events_dataset_copy['game_id'] == 2180307].head()

,id,game_id,game_away_team_id,game_away_team_name,game_competition_id,game_competition_name,game_game_date,game_home_team_id,game_home_team_name,game_matchday,...,shot_distance,shot_type,event_leading_to_shot,seq_xG,pos_xG,event_type_name,home_team_phases,home_team_phases_label,away_team_phases,away_team_phases_label


In [208]:
display_all(pre_joined_events_dataset_copy[(pre_joined_events_dataset_copy['game_id'] == 2507730) & (pre_joined_events_dataset_copy['gameClock'] > 1000)].head())


,id,game_id,game_away_team_id,game_away_team_name,game_competition_id,game_competition_name,game_game_date,game_home_team_id,game_home_team_name,game_matchday,game_season_id,game_season_name,event_id,event_event_id,event_type_id,event_period_id,event_period_id_text,event_version,event_min,event_sec,event_player_id,event_outcome,event_timestamp,qualifier,lastModified,sequence_id,f24_event_type,event_team_id,event_type_order,event_assist,event_keypass,prev_possession_id,prev_event_team_id,prev_event_period_id,event_time,possession_id,event_team_no,event_sequence_no,Period,gameClock,possession_id2,start_x,start_y,crosses,head_pass,set_play,successful_through_balls,successful_crosses,successful_passes_to_final_3rd,Player,primary_position,receiver,event_receiver_id,carry_x,carry_y,shot_y,blocked_x,blocked_y,through_balls_received,crosses_received,event_possession_id2_no,prev_end_x,prev_end_y,percent_of_carry_distance_remaining_to_goal,percent_of_pass_distance_remaining_to_goal,vertical_pass_fwd_distance,vertical_pass_distance,horizontal_pass_distance,vertical_carry_fwd_distance,vertical_carry_distance,horizontal_carry_distance,pass_distance,pass_start_pitch_zone,pass_end_pitch_zone,pass_angle,seq_shot,pos_shot,pos_passes,pos_successful_passes,event_x,event_y,end_x,end_y,EVENT,path,xG,Opta_xG,Opta_xGOT,assisted_xG,assisted_opta_xG,assisted_Big_Chance,shot_state,shot_state_detail,shot_distance,shot_type,event_leading_to_shot,seq_xG,pos_xG,event_type_name,home_team_phases,home_team_phases_label,away_team_phases,away_team_phases_label


In [209]:
#joined_events_dataset[(pre_joined_events_dataset_copy['game_id'] == 2556944) & (joined_events_dataset['gameClock'] > 1000) ].head()

In [210]:
joined_phases_dataset = pre_joined_phases_dataset.copy()
joined_events_dataset = pre_joined_events_dataset_copy[~pre_joined_events_dataset_copy['possession_id'].isnull()].reset_index()

In [211]:
joined_phases_dataset['game_id'].unique()

<IntegerArray>
[2411546, 2411547, 2411548, 2609006, 2608989, 2608993, 2608980, 2608992,
 2609008, 2608991,
 ...
 2409818, 2409830, 2409841, 2409829, 2409842, 2409817, 2409847, 2409858,
 2409810, 2409828]
Length: 209, dtype: Int64

In [212]:
joined_events_dataset['game_id'].unique()

<IntegerArray>
[2409809, 2409810, 2609009, 2609006, 2609004, 2609008, 2609010, 2609011,
 2609005, 2609007,
 ...
 2441418, 2485839, 2485886, 2486058, 2488539, 2488588, 2490038, 2490778,
 2514202, 2514201]
Length: 210, dtype: Int64

In [213]:
joined_phases_dataset[joined_phases_dataset['game_id'] == 2507730].head()

,game_id,team_id,phase,label,half,set_piece_type,gk_label,start_time,end_time,time_diff


In [214]:
display_all(joined_events_dataset[joined_events_dataset['home_team_phases_label'] != joined_events_dataset['away_team_phases_label']].head())

,index,id,game_id,game_away_team_id,game_away_team_name,game_competition_id,game_competition_name,game_game_date,game_home_team_id,game_home_team_name,game_matchday,game_season_id,game_season_name,event_id,event_event_id,event_type_id,event_period_id,event_period_id_text,event_version,event_min,event_sec,event_player_id,event_outcome,event_timestamp,qualifier,lastModified,sequence_id,f24_event_type,event_team_id,event_type_order,event_assist,event_keypass,prev_possession_id,prev_event_team_id,prev_event_period_id,event_time,possession_id,event_team_no,event_sequence_no,Period,gameClock,possession_id2,start_x,start_y,crosses,head_pass,set_play,successful_through_balls,successful_crosses,successful_passes_to_final_3rd,Player,primary_position,receiver,event_receiver_id,carry_x,carry_y,shot_y,blocked_x,blocked_y,through_balls_received,crosses_received,event_possession_id2_no,prev_end_x,prev_end_y,percent_of_carry_distance_remaining_to_goal,percent_of_pass_distance_remaining_to_goal,vertical_pass_fwd_distance,vertical_pass_distance,horizontal_pass_distance,vertical_carry_fwd_distance,vertical_carry_distance,horizontal_carry_distance,pass_distance,pass_start_pitch_zone,pass_end_pitch_zone,pass_angle,seq_shot,pos_shot,pos_passes,pos_successful_passes,event_x,event_y,end_x,end_y,EVENT,path,xG,Opta_xG,Opta_xGOT,assisted_xG,assisted_opta_xG,assisted_Big_Chance,shot_state,shot_state_detail,shot_distance,shot_type,event_leading_to_shot,seq_xG,pos_xG,event_type_name,home_team_phases,home_team_phases_label,away_team_phases,away_team_phases_label
0,0,2615642137,2409809,9138,Morocco U17,670,FIFA U17 World Cup,2023-11-10 09:00:00+00:00,9139,Panama U17,1,2023,Season 2023/2024,2615642137,3,1,1,First half,1699606880306,0,0,611477,1,2023-11-10 09:01:16.154000+00:00,"[{'id': '4645005665', 'qualifier_id': '56', 'v...",2023-11-10 09:01:21+00:00,1,1,9139,0,<NA>,<NA>,<NA>,<NA>,<NA>,0,0,2,1,1st Half,0,0a,50.0,50.0,0,0,0,0,0,0,Kevin Walder,CF,Érick Díaz,611483,NaN,NaN,NaN,NaN,NaN,0,0,1,NaN,NaN,NaN,-0.466805,-24.360,24.360,8.024,NaN,NaN,NaN,25.6,Mid 3rd Central,Def 3rd Central,161.768507,0,0,2,1,50.0,50.0,26.8,61.8,EVENT START,2,NaN,NaN,NaN,NaN,NaN,<NA>,None,None,NaN,None,None,0.0,0.0,Pass,None,None,None,None
1,1,2615642145,2409809,9138,Morocco U17,670,FIFA U17 World Cup,2023-11-10 09:00:00+00:00,9139,Panama U17,1,2023,Season 2023/2024,2615642145,4,1,1,First half,1699606882663,0,4,611483,0,2023-11-10 09:01:20.299000+00:00,"[{'id': '4645005709', 'qualifier_id': '155', '...",2023-11-10 09:01:22+00:00,1,1,9139,0,<NA>,<NA>,1,9139,1,4,0,3,2,1st Half,4,0a,26.8,61.8,0,0,0,0,0,0,Érick Díaz,CB,None,<NA>,NaN,NaN,NaN,NaN,NaN,0,0,2,26.8,61.8,NaN,0.466141,50.505,50.505,26.792,NaN,NaN,NaN,57.2,Def 3rd Central,Final 3rd Left,-27.945152,0,0,2,1,26.8,61.8,74.9,101.2,EVENT START,2,NaN,NaN,NaN,NaN,NaN,<NA>,None,None,NaN,None,None,0.0,0.0,Pass,CREATE,CREATE 1,MID BLOCK,MID BLOCK 1
2,2,2615642145,2409809,9138,Morocco U17,670,FIFA U17 World Cup,2023-11-10 09:00:00+00:00,9139,Panama U17,1,2023,Season 2023/2024,2615642145,4,1,1,First half,1699606882663,0,4,611483,0,2023-11-10 09:01:20.299000+00:00,"[{'id': '4645005709', 'qualifier_id': '155', '...",2023-11-10 09:01:22+00:00,1,1,9139,0,<NA>,<NA>,1,9139,1,4,0,3,2,1st Half,4,0a,26.8,61.8,0,0,0,0,0,0,Érick Díaz,CB,None,<NA>,NaN,NaN,NaN,NaN,NaN,0,0,2,26.8,61.8,NaN,0.466141,50.505,50.505,26.792,NaN,NaN,NaN,57.2,Def 3rd Central,Final 3rd Left,-27.945152,0,0,2,1,74.9,101.2,74.9,101.2,PASS END,3,NaN,NaN,NaN,NaN,NaN,<NA>,None,None,NaN,None,None,0.0,0.0,Pass,CREATE,CREATE 1,MID BLOCK,MID BLOCK 1
3,3,2615642179,2409809,9138,Morocco U17,670,FIFA U17 World Cup,2023-11-10 09:00:00+00:00,9139,Panama U17,1,2023,Season 2023/2024,2615642179,4,1,1,First half,1699607194078,0,8,499453,0,2023-11-10 09:01:25.122000+00:00,"[{'id': '4645005863', 'qualifier_id': '107', '...",2023-11-10 09:06:34+00:00,2,1,9138,0,<NA>,<NA>,1,9139,1,8,1,3,1,1st Half,8,1b,33.2,-1.0,0,0,1,0,0,0,Naoufel El Hannach,FB,None,<NA>,NaN,NaN,NaN,NaN,NaN,0,0,1,NaN,NaN,NaN,0.198654,11.8

In [215]:
match_phases_file_name = 'match_phases'
pre_albania_phases = joined_phases_dataset.copy()

In [216]:

pre_albania_phases['time_diff'] = pre_albania_phases['end_time'] - pre_albania_phases['start_time']

albania_phases = pre_albania_phases#[pre_albania_phases['time_diff'] >= 0]

albania_phases[albania_phases['game_id'] == 2507730][albania_phases['half'] == 2][albania_phases['start_time'] > 4714-2700].sort_values(by = ['game_id','half','start_time']).head(35)


/var/folders/2m/31gtyphx2h3fz_nycnfgx60c0000gp/T/ipykernel_23352/2016393436.py:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  albania_phases[albania_phases['game_id'] == 2507730][albania_phases['half'] == 2][albania_phases['start_time'] > 4714-2700].sort_values(by = ['game_id','half','start_time']).head(35)
/var/folders/2m/31gtyphx2h3fz_nycnfgx60c0000gp/T/ipykernel_23352/2016393436.py:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  albania_phases[albania_phases['game_id'] == 2507730][albania_phases['half'] == 2][albania_phases['start_time'] > 4714-2700].sort_values(by = ['game_id','half','start_time']).head(35)


,game_id,team_id,phase,label,half,set_piece_type,gk_label,start_time,end_time,time_diff


In [217]:

phases_describe = albania_phases.groupby(by = ['game_id','team_id','phase'])['time_diff'].describe()
phases_sum = albania_phases.groupby(by = ['game_id','team_id','phase'])['time_diff'].sum()


In [218]:
phases_count = albania_phases[['game_id','team_id','label','phase']].drop_duplicates().groupby(by = ['game_id','team_id','phase'])['label'].count()

phases_count_sum = pd.merge(phases_describe, phases_sum, on = ['game_id','team_id','phase']).reset_index()

display_all(phases_count_sum)

,game_id,team_id,phase,count,mean,std,min,25%,50%,75%,max,time_diff
0,2200777,9459,BUILD,49.0,12.571510,9.522494,0.000,5.07200,9.6160,18.92800,46.275,616.004
1,2200777,9459,CREATE,78.0,13.538231,14.521663,0.000,3.51400,9.1765,19.43800,86.864,1055.982
2,2200777,9459,FINISH,33.0,15.473061,12.689157,0.000,6.36500,12.5810,27.45500,50.622,510.611
3,2200777,9459,HIGH PRESS,51.0,7.303039,6.879728,0.000,2.83400,5.2900,10.51100,31.335,372.455
4,2200777,9459,LOW BLOCK,16.0,15.860375,24.609165,0.000,3.35900,8.0060,18.64175,101.475,253.766
...,...,...,...,...,...,...,...,...,...,...,...,...
4175,2609011,9896,MID BLOCK,53.0,8.296151,12.514334,0.000,2.00000,4.0000,7.79200,68.281,439.696
4176,2609011,9896,OPP SET PIECES,34.0,14.083441,24.920052,0.000,2.66200,7.3725,13.00000,133.000,478.837
4177,2609011,9896,SET PIECES,32.0,13.779969,12.415209,0.000,3.88525,11.5000,17.15500,48.806,440.959
4178,2609011,9896,TRANSITION A-D,31.0,8.459161,6.554380,1.344,4.00000,8.0000,9.96150,32.000,262.234


In [219]:
pre_pre_events = joined_events_dataset.copy()
filt_evented_pre_pre_events = pre_pre_events[pre_pre_events['EVENT'] == 'EVENT START']

filt_evented_pre_pre_events.Player.unique()

array(['Kevin Walder', 'Érick Díaz', 'Naoufel El Hannach', ...,
       'Amadeus Sundgren', 'Brad de Jesus', 'Will Procter'], dtype=object)

In [220]:
display_all(filt_evented_pre_pre_events.head(10))

,index,id,game_id,game_away_team_id,game_away_team_name,game_competition_id,game_competition_name,game_game_date,game_home_team_id,game_home_team_name,game_matchday,game_season_id,game_season_name,event_id,event_event_id,event_type_id,event_period_id,event_period_id_text,event_version,event_min,event_sec,event_player_id,event_outcome,event_timestamp,qualifier,lastModified,sequence_id,f24_event_type,event_team_id,event_type_order,event_assist,event_keypass,prev_possession_id,prev_event_team_id,prev_event_period_id,event_time,possession_id,event_team_no,event_sequence_no,Period,gameClock,possession_id2,start_x,start_y,crosses,head_pass,set_play,successful_through_balls,successful_crosses,successful_passes_to_final_3rd,Player,primary_position,receiver,event_receiver_id,carry_x,carry_y,shot_y,blocked_x,blocked_y,through_balls_received,crosses_received,event_possession_id2_no,prev_end_x,prev_end_y,percent_of_carry_distance_remaining_to_goal,percent_of_pass_distance_remaining_to_goal,vertical_pass_fwd_distance,vertical_pass_distance,horizontal_pass_distance,vertical_carry_fwd_distance,vertical_carry_distance,horizontal_carry_distance,pass_distance,pass_start_pitch_zone,pass_end_pitch_zone,pass_angle,seq_shot,pos_shot,pos_passes,pos_successful_passes,event_x,event_y,end_x,end_y,EVENT,path,xG,Opta_xG,Opta_xGOT,assisted_xG,assisted_opta_xG,assisted_Big_Chance,shot_state,shot_state_detail,shot_distance,shot_type,event_leading_to_shot,seq_xG,pos_xG,event_type_name,home_team_phases,home_team_phases_label,away_team_phases,away_team_phases_label
0,0,2615642137,2409809,9138,Morocco U17,670,FIFA U17 World Cup,2023-11-10 09:00:00+00:00,9139,Panama U17,1,2023,Season 2023/2024,2615642137,3,1,1,First half,1699606880306,0,0,611477,1,2023-11-10 09:01:16.154000+00:00,"[{'id': '4645005665', 'qualifier_id': '56', 'v...",2023-11-10 09:01:21+00:00,1,1,9139,0,<NA>,<NA>,<NA>,<NA>,<NA>,0,0,2,1,1st Half,0,0a,50.0,50.0,0,0,0,0,0,0,Kevin Walder,CF,Érick Díaz,611483,NaN,NaN,NaN,NaN,NaN,0,0,1,NaN,NaN,NaN,-0.466805,-24.360,24.360,8.024,NaN,NaN,NaN,25.6,Mid 3rd Central,Def 3rd Central,161.768507,0,0,2,1,50.0,50.0,26.8,61.8,EVENT START,2,NaN,NaN,NaN,NaN,NaN,<NA>,None,None,NaN,None,None,0.0,0.0,Pass,None,None,None,None
1,1,2615642145,2409809,9138,Morocco U17,670,FIFA U17 World Cup,2023-11-10 09:00:00+00:00,9139,Panama U17,1,2023,Season 2023/2024,2615642145,4,1,1,First half,1699606882663,0,4,611483,0,2023-11-10 09:01:20.299000+00:00,"[{'id': '4645005709', 'qualifier_id': '155', '...",2023-11-10 09:01:22+00:00,1,1,9139,0,<NA>,<NA>,1,9139,1,4,0,3,2,1st Half,4,0a,26.8,61.8,0,0,0,0,0,0,Érick Díaz,CB,None,<NA>,NaN,NaN,NaN,NaN,NaN,0,0,2,26.8,61.8,NaN,0.466141,50.505,50.505,26.792,NaN,NaN,NaN,57.2,Def 3rd Central,Final 3rd Left,-27.945152,0,0,2,1,26.8,61.8,74.9,101.2,EVENT START,2,NaN,NaN,NaN,NaN,NaN,<NA>,None,None,NaN,None,None,0.0,0.0,Pass,CREATE,CREATE 1,MID BLOCK,MID BLOCK 1
3,3,2615642179,2409809,9138,Morocco U17,670,FIFA U17 World Cup,2023-11-10 09:00:00+00:00,9139,Panama U17,1,2023,Season 2023/2024,2615642179,4,1,1,First half,1699607194078,0,8,499453,0,2023-11-10 09:01:25.122000+00:00,"[{'id': '4645005863', 'qualifier_id': '107', '...",2023-11-10 09:06:34+00:00,2,1,9138,0,<NA>,<NA>,1,9139,1,8,1,3,1,1st Half,8,1b,33.2,-1.0,0,0,1,0,0,0,Naoufel El Hannach,FB,None,<NA>,NaN,NaN,NaN,NaN,NaN,0,0,1,NaN,NaN,NaN,0.198654,11.865,11.865,11.492,NaN,NaN,NaN,16.5,Def 3rd Right,Mid 3rd Right,44.085092,0,0,1,0,33.2,-1.0,44.5,15.9,EVENT START,2,NaN,NaN,NaN,NaN,NaN,<NA>,None,None,NaN,None,None,0.0,0.0,Pass,None,None,None,None
5,5,2615642195,2409809,9138,Morocco U17,670,FIFA U17 World Cup,2023-11-10 09:00:00+00:00,9139,Panama U17,1,2023,Season 2023/2024,2615642195,7,1,1,First half,1699606891310,0,13,611476,0,2023-11-10 09:01:29.929000+00:00,"[{'id': '4645005923', 'qualifier_id': '56', 'v...",2023-11-10 09:01:31+00:00,3,1,9139,0,<NA>,<NA>,3,9139,1,13,2,6,2,1st Half,13,2a,55.6,81.3,0,0,0,0,0,0,Anel Ryce,CM,None,<NA>,NaN,NaN,NaN,NaN,NaN,0,0,2,NaN,NaN,NaN,0.070571,1.680,1.680,6.256,NaN,NaN

In [221]:
filt_evented_pre_pre_events.groupby(by = ['event_period_id'])['gameClock'].min()

event_period_id
1       0
2    2700
Name: gameClock, dtype: Int64

In [222]:
filt_evented_pre_pre_events[(filt_evented_pre_pre_events['game_away_team_name'].str.contains('Ireland')) |  (filt_evented_pre_pre_events['game_home_team_name'].str.contains('Ireland'))]['game_id'].unique()

<IntegerArray>
[2609011, 2556999, 2557001, 2608977, 2557002, 2441364]
Length: 6, dtype: Int64

In [223]:
import pandas as pd
import warnings

# Define the long_or_switch function
def long_or_switch(row):
    description_pass = ''
    
    if row['vertical_pass_distance'] > 25 and row['horizontal_pass_distance'] > 35:
        description_pass = "Long and Switch"
    elif row['vertical_pass_distance'] > 25:
        description_pass = "Long"
    elif row['horizontal_pass_distance'] > 35:
        description_pass = "Switch"
    else:
        description_pass = ''

    return description_pass

# Define the timings_for_create function
def timings_for_create(pre_pre_events, game_id, half, label, team_id, start_time, end_time):

    if int(half) == 2:
        start_time = start_time + 2700
        end_time = end_time + 2700

    if start_time == end_time:
        pre_pre_events_filtered = pre_pre_events[(pre_pre_events['game_id'] == game_id) & 
                                            (pre_pre_events['event_team_id'] == team_id) & 
                                             (pre_pre_events['gameClock'] == start_time)]
    else:
        pre_pre_events_filtered = pre_pre_events[(pre_pre_events['game_id'] == game_id) & 
                                            (pre_pre_events['event_team_id'] == team_id) & 
                                             (pre_pre_events['gameClock'] >= start_time) & 
                                             (pre_pre_events['gameClock'] < end_time)]
    
    pre_pre_events_filtered_s = pre_pre_events_filtered[['game_id', 'event_team_id', 'gameClock', 'event_outcome',
                                                         'pass_start_pitch_zone', 'pass_end_pitch_zone', 
                                                         'vertical_pass_distance', 'horizontal_pass_distance']]

    
    # Apply the long_or_switch function to each row
    pre_pre_events_filtered_s['description_pass'] = pre_pre_events_filtered_s.apply(lambda row: long_or_switch(row), axis=1)
    pre_pre_events_filtered_s['description_pass'] = pre_pre_events_filtered_s['description_pass'].astype(str)
    
    long_passes = len(pre_pre_events_filtered_s[pre_pre_events_filtered_s['description_pass'].str.contains('Long')])
    switch_passes = len(pre_pre_events_filtered_s[pre_pre_events_filtered_s['description_pass'].str.contains('Switch')])

    start_zones = list(map(lambda row: str(row).replace('Final 3rd ', '').replace('Mid 3rd ', '').replace('Def 3rd ', ''), 
                           pre_pre_events_filtered_s['pass_start_pitch_zone'].unique()))
    end_zones = list(map(lambda row: str(row).replace('Final 3rd ', '').replace('Mid 3rd ', '').replace('Def 3rd ', ''), 
                         pre_pre_events_filtered_s['pass_end_pitch_zone'].unique()))
    
    zones = list(filter(lambda a: a is not None, list(set(start_zones + end_zones))))

    inside_left = 0
    inside_right = 0
    side_left = 0
    side_right = 0
    central = 0
    
    for any_zone in zones:
        if 'Inside Left' in any_zone:
            inside_left += 1
        elif 'Inside Right' in any_zone:
            inside_right += 1
        elif 'Left' in any_zone:
            side_left += 1
        elif 'Right' in any_zone:
            side_right += 1
        elif 'Central' in any_zone:
            central += 1

    vertical_distance = pre_pre_events_filtered_s['vertical_pass_distance'].sum()
    horizontal_pass_distance = pre_pre_events_filtered_s['horizontal_pass_distance'].sum()
    event_outcome_sum = pre_pre_events_filtered_s['event_outcome'].sum()

    pre_pre_events_filtered_s['label'] = label
    pre_pre_events_filtered_s['team_phase_id'] = team_id
    
    outcome_df = pre_pre_events_filtered_s.groupby(by=['game_id','team_phase_id','label'])[['vertical_pass_distance', 'horizontal_pass_distance', 'event_outcome']].sum().reset_index()
    
    outcome_df['n_of_events'] = len(pre_pre_events_filtered_s)
    
    outcome_df['half'] = half
    outcome_df['start_time'] = start_time
    outcome_df['end_time'] = end_time
    
    outcome_df['left'] = side_left
    outcome_df['inside_left'] = inside_left
    outcome_df['central'] = central
    outcome_df['inside_right'] = inside_right
    outcome_df['right'] = side_right
    outcome_df['long_passes'] = long_passes
    outcome_df['switch_passes'] = switch_passes

    return outcome_df


# Assuming albania_phases DataFrame is already created and pandas is imported
albania_phases = albania_phases.drop_duplicates()

with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    
    # Filter the create phases
    check_phases = albania_phases[albania_phases['phase'].isin(['BUILD','CREATE','FINISH'])]
    
    # Apply the timings_for_create function to each row in the create phases
    updated_check_phases_zones = check_phases.apply(lambda row: timings_for_create(filt_evented_pre_pre_events, row['game_id'], row['half'], row['label'], row['team_id'], row['start_time'], row['end_time']), axis=1)
    concatenated_df = pd.concat(updated_check_phases_zones.to_list()).reset_index(drop=True).drop_duplicates()
    
display_all(concatenated_df.head(20))

,game_id,team_phase_id,label,vertical_pass_distance,horizontal_pass_distance,event_outcome,n_of_events,half,start_time,end_time,left,inside_left,central,inside_right,right,long_passes,switch_passes
0,2411546,9459,BUILD 1,26.670,74.392,6,6,1,27.835,51.947,1,1,1,1,1,0,0
1,2411547,9459,BUILD 1,6.825,11.628,1,1,1,250.544,258.560,1,0,0,0,0,0,0
2,2411548,9459,BUILD 1,29.925,43.860,1,2,1,26.174,37.743,1,0,0,1,0,1,1
3,2411548,9459,BUILD 10,39.795,41.276,4,4,1,1200.850,1221.399,0,1,1,1,0,0,0
4,2411547,9459,BUILD 10,19.950,4.896,1,2,1,1987.196,1989.996,0,0,0,0,1,0,0
5,2411546,9459,BUILD 11,66.570,70.652,3,4,1,961.621,985.087,0,1,0,1,0,1,0
6,2411547,9459,BUILD 11,12.600,4.284,1,1,1,2010.908,2020.252,0,0,1,0,0,0,0
7,2411548,9459,BUILD 11,70.980,67.932,4,4,1,1460.469,1474.564,1,1,1,0,0,1,0
8,2411546,9459,BUILD 12,5.250,40.256,2,3,1,1020.453,1037.487,0,1,1,1,0,0,0
9,2411547,9459,BUILD 12,52.500,60.792,5,6,1,2062.637,2090.374,0,0,1,1,1,0,0


In [224]:
# Define the description_of_create_phase function
def description_of_create_phase(row):
    if row['long_passes'] >= 1:
        return 'Into / Onto'
    elif row['switch_passes'] >= 1:
        return 'Switch'
    elif len(list(filter(lambda item: item > 0, [row['left'], row['inside_left'], row['central'], row['right'], row['inside_right']]))) >= 4:
        return 'Around'
    else:
        return 'Through'
    
# Apply the description_of_create_phase function to each row and assign the result to a new column
concatenated_df['phase_description'] = concatenated_df.apply(lambda row: description_of_create_phase(row), axis=1)

# Reset the index
concatenated_df = concatenated_df.reset_index(drop=True)

# Display the DataFrame
display_all(concatenated_df.sort_values(by = ['game_id','start_time']).head(10))

,game_id,team_phase_id,label,vertical_pass_distance,horizontal_pass_distance,event_outcome,n_of_events,half,start_time,end_time,left,inside_left,central,inside_right,right,long_passes,switch_passes,phase_description
5648,2200777,9459,CREATE 1,36.120,31.076,2,3,1,0.538,9.079,1,0,1,0,0,1,0,Into / Onto
5657,2200777,9459,CREATE 2,110.565,120.972,6,7,1,11.365,37.157,1,1,1,0,1,2,1,Into / Onto
5733,2200777,18187,BUILD 1,0.000,0.000,1,1,1,61.245,65.859,0,1,0,0,0,0,0,Through
5766,2200777,18187,CREATE 1,1.785,0.408,0,1,1,65.859,70.148,0,1,0,0,0,0,0,Through
5666,2200777,9459,CREATE 3,64.575,86.496,6,6,1,70.148,88.941,1,1,1,0,1,1,1,Into / Onto
5711,2200777,9459,FINISH 1,58.590,50.864,6,6,1,88.941,101.493,0,0,1,2,1,0,0,Through
5675,2200777,9459,CREATE 4,38.325,54.876,3,4,1,101.493,118.764,0,0,1,1,1,0,0,Through
5602,2200777,9459,BUILD 1,49.245,64.192,3,4,1,139.876,153.004,1,1,1,1,1,0,0,Around
5612,2200777,9459,BUILD 2,109.725,58.616,3,5,1,190.068,208.996,1,1,0,0,0,2,0,Into / Onto
5739,2200777,18187,BUILD 2,22.365,3.808,0,1,1,238.794,242.778,0,0,0,0,1,0,0,Through


In [225]:
concatenated_df[concatenated_df['game_id'].isin([2354433, 2354487, 2507730])].groupby(by = ['game_id']).count()

,team_phase_id,label,vertical_pass_distance,horizontal_pass_distance,event_outcome,n_of_events,half,start_time,end_time,left,inside_left,central,inside_right,right,long_passes,switch_passes,phase_description
game_id,,,,,,,,,,,,,,,,,


In [226]:
concatenated_df.groupby(by = ['game_id']).count()

,team_phase_id,label,vertical_pass_distance,horizontal_pass_distance,event_outcome,n_of_events,half,start_time,end_time,left,inside_left,central,inside_right,right,long_passes,switch_passes,phase_description
game_id,,,,,,,,,,,,,,,,,
2200777,213,213,213,213,213,213,213,213,213,213,213,213,213,213,213,213,213
2245009,230,230,230,230,230,230,230,230,230,230,230,230,230,230,230,230,230
2256819,205,205,205,205,205,205,205,205,205,205,205,205,205,205,205,205,205
2256820,202,202,202,202,202,202,202,202,202,202,202,202,202,202,202,202,202
2258557,188,188,188,188,188,188,188,188,188,188,188,188,188,188,188,188,188
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2609007,219,219,219,219,219,219,219,219,219,219,219,219,219,219,219,219,219
2609008,178,178,178,178,178,178,178,178,178,178,178,178,178,178,178,178,178
2609009,212,212,212,212,212,212,212,212,212,212,212,212,212,212,212,212,212


In [227]:
# Assuming df_events is your DataFrame

game_info = pre_pre_events[['game_id','game_home_team_id','game_home_team_name','game_away_team_name']].drop_duplicates()

concatenated_df_j = pd.merge(concatenated_df, game_info, on = ['game_id'])

concatenated_df_j['event_team_name'] = concatenated_df_j.apply(
    lambda row: row['game_home_team_name'] if row['game_home_team_id'] == row['team_phase_id'] else row['game_away_team_name'],
    axis=1
)
concatenated_df_j['against_team_name'] = concatenated_df_j.apply(
    lambda row: row['game_away_team_name'] if row['game_home_team_id'] == row['team_phase_id'] else row['game_home_team_name'],
    axis=1
)

concatenated_df_j = concatenated_df_j.drop(columns = {'game_home_team_id','game_home_team_name','game_away_team_name'})
albania_phases_add_info = albania_phases[['game_id','team_id','half','label','phase','time_diff']].drop_duplicates().rename(columns = {'team_id':'team_phase_id'})

concatenated_df_jj = pd.merge(concatenated_df_j, albania_phases_add_info, on = ['game_id','half','team_phase_id','label'])
concatenated_df_jjj = concatenated_df_jj[concatenated_df_jj['phase'].isin(['BUILD','CREATE','FINISH'])]
concatenated_df_jjj = concatenated_df_jjj.rename(columns = {'label':'create_phase_label'})

display_all(concatenated_df_jjj.sort_values(by = ['game_id','start_time']).head(10))

,game_id,team_phase_id,create_phase_label,vertical_pass_distance,horizontal_pass_distance,event_outcome,n_of_events,half,start_time,end_time,left,inside_left,central,inside_right,right,long_passes,switch_passes,phase_description,event_team_name,against_team_name,phase,time_diff
5648,2200777,9459,CREATE 1,36.120,31.076,2,3,1,0.538,9.079,1,0,1,0,0,1,0,Into / Onto,England U18,Wales U18,CREATE,8.541
5657,2200777,9459,CREATE 2,110.565,120.972,6,7,1,11.365,37.157,1,1,1,0,1,2,1,Into / Onto,England U18,Wales U18,CREATE,25.792
5733,2200777,18187,BUILD 1,0.000,0.000,1,1,1,61.245,65.859,0,1,0,0,0,0,0,Through,Wales U18,England U18,BUILD,4.614
5766,2200777,18187,CREATE 1,1.785,0.408,0,1,1,65.859,70.148,0,1,0,0,0,0,0,Through,Wales U18,England U18,CREATE,4.289
5666,2200777,9459,CREATE 3,64.575,86.496,6,6,1,70.148,88.941,1,1,1,0,1,1,1,Into / Onto,England U18,Wales U18,CREATE,18.793
5711,2200777,9459,FINISH 1,58.590,50.864,6,6,1,88.941,101.493,0,0,1,2,1,0,0,Through,England U18,Wales U18,FINISH,12.552
5675,2200777,9459,CREATE 4,38.325,54.876,3,4,1,101.493,118.764,0,0,1,1,1,0,0,Through,England U18,Wales U18,CREATE,17.271
5602,2200777,9459,BUILD 1,49.245,64.192,3,4,1,139.876,153.004,1,1,1,1,1,0,0,Around,England U18,Wales U18,BUILD,13.128
5612,2200777,9459,BUILD 2,109.725,58.616,3,5,1,190.068,208.996,1,1,0,0,0,2,0,Into / Onto,England U18,Wales U18,BUILD,18.928
5739,2200777,18187,BUILD 2,22.365,3.808,0,1,1,238.794,242.778,0,0,0,0,1,0,0,Through,Wales U18,England U18,BUILD,3.984


In [228]:
concatenated_df_jjj[['game_id','team_phase_id','phase','create_phase_label','time_diff']].drop_duplicates().groupby(by = ['game_id','team_phase_id','phase'])['time_diff'].sum()

game_id  team_phase_id  phase 
2200777  9459           BUILD      614.164
                        CREATE    1017.954
                        FINISH     430.967
         18187          BUILD      338.170
                        CREATE     469.936
                                    ...   
2609011  9399           CREATE     425.011
                        FINISH     417.349
         9896           BUILD      297.419
                        CREATE     635.057
                        FINISH     424.574
Name: time_diff, Length: 1254, dtype: float64

In [229]:
concatenated_df_jjj.groupby(by = ['game_id','team_phase_id','phase_description']).count()

create_phase_label  \
game_id team_phase_id phase_description                       
2200777 9459          Around                             19   
                      Into / Onto                        39   
                      Switch                              7   
                      Through                            66   
        18187         Around                              2   
...                                                     ...   
2609011 9399          Through                            63   
        9896          Around                             16   
                      Into / Onto                        19   
                      Switch                              5   
                      Through                            71   

                                         vertical_pass_distance  \
game_id team_phase_id phase_description                           
2200777 9459          Around                                 19   
                      Into / Onto                            39   
                      Switch                                  7   
                      Through                                66   
        18187         Around                                  2   
...                                                         ...   
2609011 9399          Through                                63   
        9896          Around                                 16   
                      Into / Onto                            19   
                      Switch                                  5   
                      Through                                71   

                                         horizontal_pass_distance  \
game_id team_phase_id phase_description                             
2200777 9459          Around                                   19   
                      Into / Onto                              39   
                      Switch                                    7   
                      Through                                  66   
        18187         Around                                    2   
...                                                           ...   
2609011 9399          Through                                  63   
        9896          Around                                   16   
                      Into / Onto                              19   
                      Switch                                    5   
                      Through                                  71   

                                         event_outcome  n_of_events  half  \
game_id team_phase_id phase_description                                     
2200777 9459          Around                        19           19    19   
                      Into / Onto                   39           39    39   
                      Switch                         7            7     7   
                      Through                       66           66    66   
        18187         Around                         2            2     2   
...                                                ...          ...   ...   
2609011 9399          Through                       63           63    63   
        9896          Around                        16           16    16   
                      Into / Onto                   19           19    19   
                      Switch                         5            5     5   
                      Through                       71           71    71   

                                         start_time  end_time  left  \
game_id team_phase_id phase_description                               
2200777 9459          Around                     19        19    19   
                      Into / Onto                39        39    39   
                      Switch                      7         7     7   
                      Through                    66        66    66   
        18187         A

In [230]:
import pandas as pd

def mark_phases(df):
    df['previous_phase'] = None
    df['entering_phase'] = None
    df['event_team_name'] = None
    df['entering_phase_label'] = None
    df['event_team_name'] = None
    df['against_team_name'] = None
    
    for i in range(1, len(df)):
        if df.loc[i, 'possession_id'] == df.loc[i-1, 'possession_id']:
            if df.loc[i, 'event_team_id'] == df.loc[i, 'game_home_team_id']:
                if df.loc[i, 'home_team_phases'] != df.loc[i-1, 'home_team_phases']:
                    df.loc[i-1, 'entering_phase'] = df.loc[i, 'home_team_phases']
                    df.loc[i-1, 'entering_phase_label'] = df.loc[i, 'home_team_phases_label']
                    df.loc[i-1, 'previous_phase_label'] = df.loc[i-1, 'home_team_phases_label']
                    df.loc[i-1, 'previous_phase'] = df.loc[i-1, 'home_team_phases']
                    df.loc[i-1, 'event_team_name'] = df.loc[i-1, 'game_home_team_name']
                    df.loc[i-1, 'against_team_name'] = df.loc[i-1, 'game_away_team_name']
            elif df.loc[i, 'event_team_id'] == df.loc[i, 'game_away_team_id']:
                if df.loc[i, 'away_team_phases'] != df.loc[i-1, 'away_team_phases']:
                    df.loc[i-1, 'entering_phase'] = df.loc[i, 'away_team_phases']
                    df.loc[i-1, 'entering_phase_label'] = df.loc[i, 'away_team_phases_label']
                    df.loc[i-1, 'previous_phase_label'] = df.loc[i-1, 'away_team_phases_label']
                    df.loc[i-1, 'previous_phase'] = df.loc[i-1, 'away_team_phases']
                    df.loc[i-1, 'event_team_name'] = df.loc[i-1, 'game_away_team_name']
                    df.loc[i-1, 'against_team_name'] = df.loc[i-1, 'game_home_team_name']
    
    return df
            
def create_end_zone(end_x, end_y):
    if end_x <= 33.33:
        x_zone = 'Def 3rd'
    elif end_x <= 66.67:
        x_zone = 'Mid 3rd'
    else:
        x_zone = 'Final 3rd'
        
    if end_y <= 20:
        y_zone = 'Right'
    elif end_y <= 40:
        y_zone = 'Inside Right'
    elif end_y <= 60:
        y_zone = 'Central'
    elif end_y <= 80:
        y_zone = 'Inside Left'
    else:
        y_zone = 'Left'
    
    return f'{x_zone} {y_zone}'

def evaluate_phase_change(pre_df):
    final_out = pd.DataFrame()
    game_ids = pre_df.game_id.unique()

    for game_id in game_ids:
        df = pre_df[pre_df['game_id'] == game_id].sort_values(by=['game_id', 'gameClock', 'path']).reset_index(drop=True)
        df['long_or_switch_flag'] = ''
        df['through_around_flag'] = ''
        df['check_type'] = ''

        if df.empty:
            continue  # Skip if the DataFrame is empty

        for i in range(len(df)):
            team_id = df.iloc[i]['event_team_id']  # Use .iloc for positional indexing
            
            if df.iloc[i]['EVENT'] in ['EVENT START', 'CARRY START', 'PASS END']:
                start_zone = df.iloc[i]['pass_start_pitch_zone']
                if df.iloc[i]['EVENT'] == 'CARRY START' or (df.iloc[i]['event_type_name'] == 'Take On'):
                    end_zone = create_end_zone(df.iloc[i]['end_x'], df.iloc[i]['end_y'])
                    df.at[i, 'pass_end_pitch_zone'] = end_zone
                else:
                    end_zone = df.iloc[i]['pass_end_pitch_zone']
                    
                df.at[i, 'check_type'] = 'Regular'

                if pd.isnull(start_zone) or pd.isnull(end_zone):
                    # Handle PassEnd_BallRecovery
                    if (df.iloc[i]['EVENT'] == 'PASS_END') or (df.iloc[i]['event_type_name'] == 'Ball Recovery') and i + 1 < len(df):
                        for k in range(i + 1, len(df)):
                            next_event = df.iloc[k]
                            if next_event['event_team_id'] == df.iloc[i]['event_team_id'] and next_event['EVENT'] == 'EVENT START':
                                df.at[i, 'end_x'] = next_event['event_x']
                                df.at[i, 'end_y'] = next_event['event_y']
                                df.at[i, 'vertical_pass_distance'] = df.at[i, 'end_x'] - df.at[i, 'event_x']
                                df.at[i, 'horizontal_pass_distance'] = df.at[i, 'end_y'] - df.at[i, 'event_y']
                                end_zone = create_end_zone(next_event['event_x'], next_event['event_y'])
                                df.at[i, 'pass_end_pitch_zone'] = end_zone
                                break
                        df.at[i, 'check_type'] = 'PassEnd_BallRecovery'
                    
                    # Other checks follow (Out, No Out, etc.), similar to your original logic
                
                # Flagging long or switch passes
                long_pass = df.at[i, 'vertical_pass_distance'] >= 25
                switch_pass = df.at[i, 'horizontal_pass_distance'] >= 35

                if long_pass and switch_pass:
                    df.at[i, 'long_or_switch_flag'] = 'Long / Switch'
                elif long_pass:
                    df.at[i, 'long_or_switch_flag'] = 'Long'
                elif switch_pass:
                    df.at[i, 'long_or_switch_flag'] = 'Switch'
                    
                # Flagging through/around passes
                if pd.isnull(start_zone) or pd.isnull(end_zone):
                    df.at[i, 'through_around_flag'] = 'Error'
                else:
                    if 'Inside' in start_zone:
                        if 'Penalty Box' not in end_zone and 'Centr' not in end_zone and 'Inside' not in end_zone:
                            df.at[i, 'through_around_flag'] = 'Around'
                        elif 'Penalty Box' in end_zone or 'Centr' in end_zone or 'Inside' in end_zone:
                            df.at[i, 'through_around_flag'] = 'Through Inside'
                    elif 'Centr' in start_zone or 'Penalty Box' in start_zone:
                        if 'Penalty Box' not in end_zone and 'Centr' not in end_zone and 'Inside' not in end_zone:
                            df.at[i, 'through_around_flag'] = 'Through Inside - Outside'
                        elif 'Penalty Box' in end_zone or 'Centr' in end_zone or 'Inside' in end_zone:
                            df.at[i, 'through_around_flag'] = 'Through Inside'
                    elif 'Penalty Box' not in start_zone and 'Inside' not in start_zone and 'Centr' not in start_zone:
                        if 'Penalty Box' not in end_zone and 'Centr' not in end_zone and 'Inside' not in end_zone:
                            df.at[i, 'through_around_flag'] = 'Around'
                        elif 'Penalty Box' in end_zone or 'Centr' in end_zone or 'Inside' in end_zone:
                            df.at[i, 'through_around_flag'] = 'Through Outside - Inside'

        final_out = pd.concat([final_out, df], axis=0).reset_index(drop=True)
    
    return final_out

# Example Usage
pre_events_marked = mark_phases(pre_pre_events)  

pre_events = evaluate_phase_change(pre_events_marked)

pre_events_marked.groupby(by = ['entering_phase']).count()

,index,id,game_id,game_away_team_id,game_away_team_name,game_competition_id,game_competition_name,game_game_date,game_home_team_id,game_home_team_name,...,event_type_name,home_team_phases,home_team_phases_label,away_team_phases,away_team_phases_label,previous_phase,event_team_name,entering_phase_label,against_team_name,previous_phase_label
entering_phase,,,,,,,,,,,,,,,,,,,,,
BUILD,11114,11114,11114,11114,11114,11114,11114,11114,11114,11114,...,11114,8512,8512,8528,8528,8528,11114,11114,11114,8528
CREATE,16968,16968,16968,16968,16968,16968,16968,16968,16968,16968,...,16968,11782,11782,11815,11815,11815,16968,16968,16968,11815
FINISH,8245,8245,8245,8245,8245,8245,8245,8245,8245,8245,...,8245,4918,4918,4956,4956,4955,8245,8245,8245,4955
HIGH PRESS,152,152,152,152,152,152,152,152,152,152,...,152,140,140,141,141,140,152,152,152,140
LOW BLOCK,364,364,364,364,364,364,364,364,364,364,...,364,325,325,325,325,325,364,364,364,325
MID BLOCK,397,397,397,397,397,397,397,397,397,397,...,397,375,375,376,376,375,397,397,397,375
OPP SET PIECES,328,328,328,328,328,328,328,328,328,328,...,328,312,312,312,312,312,328,328,328,312
SET PIECES,2221,2221,2221,2221,2221,2221,2221,2221,2221,2221,...,2221,1847,1847,1851,1851,1850,2221,2221,2221,1850
TRANSITION A-D,273,273,273,273,273,273,273,273,273,273,...,273,267,267,267,267,267,273,273,273,267


In [231]:

# Filter rows where previous_phase and entering_phase are 'BUILD', 'CREATE', or 'FINISH'
valid_phases = ['BUILD', 'CREATE', 'FINISH']

phase_changes = pre_events[(pre_events['previous_phase'].isin(valid_phases)) & 
                           (pre_events['entering_phase'].isin(valid_phases))]

# Create the new DataFrame with specified columns
new_df = phase_changes[['game_id', 'event_id', 'game_home_team_name', 'game_away_team_name', 
                        'event_team_name', 'gameClock', 'event_period_id','event_min', 'event_sec', 'possession_id', 
                        'Player', 'receiver', 'EVENT', 'event_x', 'event_y', 'end_x', 'end_y', 
                        'vertical_carry_distance', 'horizontal_carry_distance', 'pass_distance', 
                        'pass_start_pitch_zone', 'pass_end_pitch_zone', 'event_type_name', 'previous_phase', 
                        'previous_phase_label', 'entering_phase','entering_phase_label', 'long_or_switch_flag', 'through_around_flag','check_type']]

# Fix event_type_name and phase_switch_mode
new_df['phase_switch_mode'] = new_df.apply(
    lambda row: f"Carry - {row['through_around_flag']}" if 'CARRY' in row['EVENT'] 
    else row['long_or_switch_flag'] if row['long_or_switch_flag'] != ''
    else row['through_around_flag'], axis=1
)

new_df['phase_change'] = new_df['previous_phase'] + ' - ' + new_df['entering_phase'] 

new_df['event_type_name'] = new_df.apply(
    lambda row: 'Carry' if 'CARRY' in row['EVENT'] else row['event_type_name'], axis=1
)

# Add progression_flag column
new_df['progression_flag'] = new_df.apply(
    lambda row: 'Progression' if (row['previous_phase'] == 'BUILD' and row['entering_phase'] in ['CREATE', 'FINISH']) 
                                 or (row['previous_phase'] == 'CREATE' and row['entering_phase'] == 'FINISH') 
    else 'Regression', axis=1
)

new_df_f = new_df[~new_df['event_type_name'].isin(['Card','Out','Ball Touch'])]#.drop(columns = {'EVENT'})

# Display the first few rows of the new DataFrame
display_all(new_df_f.groupby(by = ['event_type_name']).count())


/var/folders/2m/31gtyphx2h3fz_nycnfgx60c0000gp/T/ipykernel_23352/793366029.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['phase_switch_mode'] = new_df.apply(
/var/folders/2m/31gtyphx2h3fz_nycnfgx60c0000gp/T/ipykernel_23352/793366029.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['phase_change'] = new_df['previous_phase'] + ' - ' + new_df['entering_phase']
/var/folders/2m/31gtyphx2h3fz_nycnfgx60c0000gp/T/ipykernel_23352/793366029.py:24: SettingWithCopyWarning: 
A value is trying to

,game_id,event_id,game_home_team_name,game_away_team_name,event_team_name,gameClock,event_period_id,event_min,event_sec,possession_id,Player,receiver,EVENT,event_x,event_y,end_x,end_y,vertical_carry_distance,horizontal_carry_distance,pass_distance,pass_start_pitch_zone,pass_end_pitch_zone,previous_phase,previous_phase_label,entering_phase,entering_phase_label,long_or_switch_flag,through_around_flag,check_type,phase_switch_mode,phase_change,progression_flag
event_type_name,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Aerial,262,262,262,262,262,262,262,262,262,262,262,0,262,262,262,0,0,0,0,0,262,0,262,262,262,262,262,262,262,262,262,262
Attempt Saved,64,64,64,64,64,64,64,64,64,64,64,0,64,64,64,0,0,31,31,0,64,0,64,64,64,64,64,64,64,64,64,64
Attempted Tackle,1,1,1,1,1,1,1,1,1,1,1,0,1,1,1,0,0,0,0,0,1,0,1,1,1,1,1,1,1,1,1,1
Ball Recovery,68,68,68,68,68,68,68,68,68,68,68,0,68,68,68,68,68,10,10,0,68,68,68,68,68,68,68,68,68,68,68,68
Blocked Pass,13,13,13,13,13,13,13,13,13,13,13,0,13,13,13,0,0,0,0,0,13,0,13,13,13,13,13,13,13,13,13,13
Carry,3067,3067,3067,3067,3067,3067,3067,3067,3067,3067,3064,2414,3067,3067,3067,3067,3067,3067,3067,2729,3067,3067,3067,3067,3067,3067,3067,3067,3067,3067,3067,3067
Clearance,1,1,1,1,1,1,1,1,1,1,1,0,1,1,1,1,1,0,0,1,1,1,1,1,1,1,1,1,1,1,1,1
Corner Awarded,20,20,20,20,20,20,20,20,20,20,20,0,20,20,20,0,0,0,0,0,20,0,20,20,20,20,20,20,20,20,20,20
Coverage Interruption,1,1,1,1,1,1,1,1,1,1,0,0,1,1,1,0,0,1,1,0,1,0,1,1,1,1,1,1,1,1,1,1


In [232]:
def clean_df_after_pivot(df):

    df.columns = ['_'.join(col).strip() if isinstance(col, tuple) else col for col in df.columns.values]

    for col in df.columns:
        if col[-1] == '_':
            df = df.rename(columns = {col: col[:-1]})
        elif 'phase_switch_mode_' in col:
            df = df.rename(columns = {col: col.replace('phase_switch_mode_','').replace(' - ','_')})
        elif ('event_x' in col) or ('event_y' in col) or ('end_x' in col) or ('end_y' in col)  :
            df = df.rename(columns = {col: col.replace(' - ','_')})
       
    return df
    
def pivot_operation(df,col):

    previous_phases = new_df_f[['game_id','event_team_name',col,'phase_switch_mode','phase_change','event_x','event_y','end_x','end_y']].drop_duplicates()

    pivoted_previous_phases = clean_df_after_pivot(pd.pivot_table(previous_phases, index = ['game_id','event_team_name',col], columns = ['phase_change'], values = ['phase_switch_mode'], aggfunc = 'last').reset_index())
    event_x_pivoted_previous_phases = clean_df_after_pivot(pd.pivot_table(previous_phases, index = ['game_id','event_team_name',col], columns = ['phase_change'], values = ['event_x'], aggfunc = 'last').reset_index())
    event_y_pivoted_previous_phases = clean_df_after_pivot(pd.pivot_table(previous_phases, index = ['game_id','event_team_name',col], columns = ['phase_change'], values = ['event_y'], aggfunc = 'last').reset_index())
    end_x_pivoted_previous_phases = clean_df_after_pivot(pd.pivot_table(previous_phases, index = ['game_id','event_team_name',col], columns = ['phase_change'], values = ['end_x'], aggfunc = 'last').reset_index())
    end_y_pivoted_previous_phases = clean_df_after_pivot(pd.pivot_table(previous_phases, index = ['game_id','event_team_name',col], columns = ['phase_change'], values = ['end_y'], aggfunc = 'last').reset_index())
    
    
    join_all_1 = pd.merge(pivoted_previous_phases, event_x_pivoted_previous_phases, on = ['game_id','event_team_name',col], how = 'left')
    join_all_2 = pd.merge(join_all_1, event_y_pivoted_previous_phases, on = ['game_id','event_team_name',col], how = 'left')
    join_all_3 = pd.merge(join_all_2, end_x_pivoted_previous_phases, on = ['game_id','event_team_name',col], how = 'left')
    join_all_4 = pd.merge(join_all_3, end_y_pivoted_previous_phases, on = ['game_id','event_team_name',col], how = 'left')
    
    return join_all_4

pivoted_previous_phases = pivot_operation(new_df_f, 'previous_phase_label')
pivoted_entering_phases = pivot_operation(new_df_f, 'entering_phase_label')


In [233]:
# Rename the label columns to a common name for concatenation
pivoted_previous_phases = pivoted_previous_phases.rename(columns={'previous_phase_label': 'create_phase_label'})
pivoted_entering_phases = pivoted_entering_phases.rename(columns={'entering_phase_label': 'create_phase_label'})

# Combine the dataframes
combined_df = pd.concat([pivoted_previous_phases, pivoted_entering_phases], axis=0)

# Group by the common columns and take the first non-NaN value for each group
combined_df = combined_df.groupby(['game_id', 'event_team_name', 'create_phase_label']).apply(lambda x: x.ffill().bfill().iloc[0]).reset_index(drop=True)

# Replace NaN values in the last four columns with empty strings
for col in combined_df.columns[-4:]:
    combined_df[col] = combined_df[col].fillna('')
    
display_all(combined_df)

/var/folders/2m/31gtyphx2h3fz_nycnfgx60c0000gp/T/ipykernel_23352/1715168950.py:9: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  combined_df = combined_df.groupby(['game_id', 'event_team_name', 'create_phase_label']).apply(lambda x: x.ffill().bfill().iloc[0]).reset_index(drop=True)
/var/folders/2m/31gtyphx2h3fz_nycnfgx60c0000gp/T/ipykernel_23352/1715168950.py:9: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  combined_df = combined_df.groupby(['game_id', 'event_team_name', 'create_ph

,game_id,event_team_name,create_phase_label,BUILD_CREATE,BUILD_FINISH,CREATE_BUILD,CREATE_FINISH,FINISH_BUILD,FINISH_CREATE,event_x_BUILD_CREATE,event_x_BUILD_FINISH,event_x_CREATE_BUILD,event_x_CREATE_FINISH,event_x_FINISH_BUILD,event_x_FINISH_CREATE,event_y_BUILD_CREATE,event_y_BUILD_FINISH,event_y_CREATE_BUILD,event_y_CREATE_FINISH,event_y_FINISH_BUILD,event_y_FINISH_CREATE,end_x_BUILD_CREATE,end_x_BUILD_FINISH,end_x_CREATE_BUILD,end_x_CREATE_FINISH,end_x_FINISH_BUILD,end_x_FINISH_CREATE,end_y_BUILD_CREATE,end_y_BUILD_FINISH,end_y_CREATE_BUILD,end_y_CREATE_FINISH,end_y_FINISH_BUILD,end_y_FINISH_CREATE
0,2200777,England U18,BUILD 11,Around,NaN,NaN,NaN,NaN,NaN,45.9,NaN,NaN,NaN,NaN,NaN,97.4,NaN,NaN,NaN,NaN,NaN,65.0,NaN,NaN,NaN,NaN,NaN,93.9,NaN,,,,
1,2200777,England U18,BUILD 16,Around,NaN,NaN,NaN,NaN,NaN,41.3,NaN,NaN,NaN,NaN,NaN,87.7,NaN,NaN,NaN,NaN,NaN,39.7,NaN,NaN,NaN,NaN,NaN,99.2,NaN,,,,
2,2200777,England U18,BUILD 20,Long,NaN,NaN,NaN,NaN,NaN,39.5,NaN,NaN,NaN,NaN,NaN,38.6,NaN,NaN,NaN,NaN,NaN,99.3,NaN,NaN,NaN,NaN,NaN,17.7,NaN,,,,
3,2200777,England U18,BUILD 23,Through Inside,NaN,NaN,NaN,NaN,NaN,52.0,NaN,NaN,NaN,NaN,NaN,62.3,NaN,NaN,NaN,NaN,NaN,38.5,NaN,NaN,NaN,NaN,NaN,60.8,NaN,,,,
4,2200777,England U18,BUILD 26,Carry - Around,NaN,NaN,NaN,NaN,NaN,36.4,NaN,NaN,NaN,NaN,NaN,5.9,NaN,NaN,NaN,NaN,NaN,48.4,NaN,NaN,NaN,NaN,NaN,5.8,NaN,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17316,2609011,Switzerland U17,CREATE 46,Carry - Around,NaN,NaN,NaN,NaN,NaN,66.5,NaN,NaN,NaN,NaN,NaN,13.7,NaN,NaN,NaN,NaN,NaN,66.7,NaN,NaN,NaN,NaN,NaN,9.2,NaN,,,,
17317,2609011,Switzerland U17,CREATE 49,Around,NaN,NaN,NaN,NaN,NaN,39.3,NaN,NaN,NaN,NaN,NaN,10.6,NaN,NaN,NaN,NaN,NaN,34.3,NaN,NaN,NaN,NaN,NaN,17.7,NaN,,,,
17318,2609011,Switzerland U17,CREATE 56,Around,NaN,NaN,NaN,NaN,NaN,66.7,NaN,NaN,NaN,NaN,NaN,26.4,NaN,NaN,NaN,NaN,NaN,71.1,NaN,NaN,NaN,NaN,NaN,18.8,NaN,,,,
17319,2609011,Switzerland U17,FINISH 13,NaN,NaN,NaN,Through Inside,NaN,Around,NaN,NaN,NaN,93.0,NaN,55.0,NaN,NaN,NaN,72.2,NaN,3.8,NaN,NaN,NaN,82.0,NaN,47.0,NaN,NaN,,31.3,,7.4


In [234]:
display_all(combined_df.groupby(by = ['event_team_name']).count())

,game_id,create_phase_label,BUILD_CREATE,BUILD_FINISH,CREATE_BUILD,CREATE_FINISH,FINISH_BUILD,FINISH_CREATE,event_x_BUILD_CREATE,event_x_BUILD_FINISH,event_x_CREATE_BUILD,event_x_CREATE_FINISH,event_x_FINISH_BUILD,event_x_FINISH_CREATE,event_y_BUILD_CREATE,event_y_BUILD_FINISH,event_y_CREATE_BUILD,event_y_CREATE_FINISH,event_y_FINISH_BUILD,event_y_FINISH_CREATE,end_x_BUILD_CREATE,end_x_BUILD_FINISH,end_x_CREATE_BUILD,end_x_CREATE_FINISH,end_x_FINISH_BUILD,end_x_FINISH_CREATE,end_y_BUILD_CREATE,end_y_BUILD_FINISH,end_y_CREATE_BUILD,end_y_CREATE_FINISH,end_y_FINISH_BUILD,end_y_FINISH_CREATE
event_team_name,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Argentina U17,557,557,274,16,67,232,7,54,274,16,67,232,7,54,274,16,67,232,7,54,237,14,65,196,6,46,237,14,557,557,557,557
Australia U18,41,41,20,2,10,12,0,0,20,2,10,12,0,0,20,2,10,12,0,0,20,2,10,12,0,0,20,2,41,41,41,41
Austria U17,231,231,116,10,27,73,4,32,116,10,27,73,4,32,116,10,27,73,4,32,110,10,27,61,2,28,110,10,231,231,231,231
Austria U18,26,26,10,4,4,8,0,2,10,4,4,8,0,2,10,4,4,8,0,2,10,4,4,8,0,2,10,4,26,26,26,26
Belgium U17,220,220,120,16,46,76,4,16,120,16,46,76,4,16,120,16,46,76,4,16,120,16,44,64,4,14,120,16,220,220,220,220
Belgium U18,173,173,82,4,44,44,4,22,82,4,44,44,4,22,82,4,44,44,4,22,80,4,44,44,4,20,80,4,173,173,173,173
Bolivia U17,97,97,60,4,6,25,0,8,60,4,6,25,0,8,60,4,6,25,0,8,57,2,6,23,0,8,57,2,97,97,97,97
Bosnia and Herzegovina U18,30,30,14,2,10,4,0,4,14,2,10,4,0,4,14,2,10,4,0,4,14,0,10,4,0,4,14,0,30,30,30,30
Brazil U17,558,558,274,26,74,222,4,28,274,26,74,222,4,28,274,26,74,222,4,28,260,22,74,194,4,26,260,22,558,558,558,558


In [235]:
display_all(concatenated_df_jjj.groupby(by = ['phase']).count())

,game_id,team_phase_id,create_phase_label,vertical_pass_distance,horizontal_pass_distance,event_outcome,n_of_events,half,start_time,end_time,left,inside_left,central,inside_right,right,long_passes,switch_passes,phase_description,event_team_name,against_team_name,time_diff
phase,,,,,,,,,,,,,,,,,,,,,
BUILD,13170,13170,13170,13170,13170,13170,13170,13170,13170,13170,13170,13170,13170,13170,13170,13170,13170,13170,13170,13170,13170
CREATE,19615,19615,19615,19615,19615,19615,19615,19615,19615,19615,19615,19615,19615,19615,19615,19615,19615,19615,19615,19615,19615
FINISH,9772,9772,9772,9772,9772,9772,9772,9772,9772,9772,9772,9772,9772,9772,9772,9772,9772,9772,9772,9772,9772


In [236]:
def find_per_game_time_metrics(data, team_name):

    new_df_f_out_teamed = data[(data['event_team_name'] == team_name)]
    new_df_f_out_opponents = data[(data['against_team_name'] == team_name)]

    n_of_matches = len(new_df_f_out_teamed['game_id'].unique())
    in_possession_time_spent_per_team_phase = new_df_f_out_teamed.groupby(by = ['event_team_name','game_id','against_team_name','team_tier','against_tier', 'phase'])['time_diff'].sum().reset_index()
    pre_out_of_possession_time_spent_per_team_phase = new_df_f_out_opponents.groupby(by = ['against_team_name','game_id','event_team_name','team_tier','against_tier', 'phase'])['time_diff'].sum().reset_index()
    pre2_out_of_possession_time_spent_per_team_phase = pre_out_of_possession_time_spent_per_team_phase.rename(columns = {'against_team_name':'aaa','against_tier':'TTT'})
    pre3_out_of_possession_time_spent_per_team_phase = pre2_out_of_possession_time_spent_per_team_phase.rename(columns = {'event_team_name':'against_team_name','team_tier':'against_tier'})
    out_of_possession_time_spent_per_team_phase = pre3_out_of_possession_time_spent_per_team_phase.rename(columns = {'aaa':'event_team_name','TTT':'team_tier'})
    
    out_of_possession_time_spent_per_team_phase['phase'] = out_of_possession_time_spent_per_team_phase['phase'].replace({
        'BUILD': '% IN HIGH BLOCK',
        'CREATE': '% IN MID BLOCK',
        'FINISH': '% IN LOW BLOCK'
    })

    out_of_possession_time_spent_per_team_phase = out_of_possession_time_spent_per_team_phase[in_possession_time_spent_per_team_phase.columns]
    
    time_spent_metrics = pd.concat([in_possession_time_spent_per_team_phase, out_of_possession_time_spent_per_team_phase])
    
    time_spent_metrics['KPI'] = time_spent_metrics['phase'].replace({
        'BUILD': '% IN BUILD PHASE',
        'CREATE': '% IN CREATE PHASE',
        'FINISH': '% IN FINISH PHASE'
    })
    
    time_spent_metrics['Value'] = (time_spent_metrics['time_diff'] / time_spent_metrics.groupby(by = ['game_id','against_team_name','event_team_name'])['time_diff'].transform('sum') * 100).round(2)
    
    teamed_time_metrics = time_spent_metrics[(time_spent_metrics['event_team_name'] == team_name)]
    teamed_time_metrics['n_of_matches'] = n_of_matches
    
    return teamed_time_metrics
    
'''
def find_per_game_create_to_finish_metrics(data, team_name):

    new_df_f_out_teamed = data[(data['event_team_name'] == team_name)]
    new_df_f_out_opponents = data[(data['against_team_name'] == team_name)]

    n_of_matches = len(new_df_f_out_teamed['game_id'].unique())
    create_to_finish_successful = new_df_f_out_teamed[(new_df_f_out_teamed['CREATE_FINISH'] != '') & (~new_df_f_out_teamed['CREATE_FINISH'].isnull())].groupby(by = ['event_team_name','against_team_name','game_id'])['game_phase_id'].count().reset_index().rename(columns = {'game_phase_id':'successful_create_to_finish'})
    create_to_finish_total = new_df_f_out_teamed.groupby(by = ['event_team_name','against_team_name','game_id'])['game_phase_id'].count().reset_index().rename(columns = {'game_phase_id':'total_create_to_finish'})
    create_to_finish_metrics = pd.merge(create_to_finish_total, create_to_finish_successful, on = ['event_team_name','against_team_name','game_id'])
    
    opp_n_of_matches = len(new_df_f_out_opponents['game_id'].unique())
    opp_create_to_finish_successful = new_df_f_out_opponents[(new_df_f_out_opponents['CREATE_FINISH'] != '') & (~new_df_f_out_opponents['CREATE_FINISH'].isnull())].groupby(by = ['against_team_name','event_team_name','game_id'])['game_phase_id'].count().reset_index().rename(columns = {'game_phase_id':'successful_create_to_finish'})
    opp_create_to_finish_total = new_df_f_out_opponents.groupby(by = ['against_team_name','event_team_name','game_id'])['game_phase_id'].count().reset_index().rename(columns = {'game_phase_id':'total_create_to_finish'})
    opp_create_to_finish_metrics = pd.merge(opp_create_to_finish_total, opp_create_to_finish_successful, on = ['against_team_name','event_team_name','game_id'])
    
    teamed_create_to_finish_metrics = create_to_finish_metrics[(create_to_finish_metrics['event_team_name'] == team_name)]
    opp_create_to_finish_metrics = opp_create_to_finish_metrics[(opp_create_to_finish_metrics['against_team_name'] == team_name)]

    teamed_create_to_finish_metrics['Value'] = (teamed_create_to_finish_metrics['successful_create_to_finish']  / teamed_create_to_finish_metrics['total_create_to_finish']  * 100).round(2)
    opp_create_to_finish_metrics['Value'] = (opp_create_to_finish_metrics['successful_create_to_finish']  / opp_create_to_finish_metrics['total_create_to_finish']  * 100).round(2)
    
    teamed_create_to_finish_metrics['KPI'] = 'Create to Finish Success Rate (%)'
    teamed_create_to_finish_metrics['n_of_matches'] = n_of_matches

    opp_create_to_finish_metrics['KPI'] = 'Opponent Create to Finish Success Rate (%)'
    opp_create_to_finish_metrics['n_of_matches'] = opp_n_of_matches

    opp_create_to_finish_metrics_p = opp_create_to_finish_metrics.rename(columns = {'against_team_name':'ttt'})
    opp_create_to_finish_metrics_p2 = opp_create_to_finish_metrics_p.rename(columns = {'event_team_name':'against_team_name'})
    opp_create_to_finish_metrics_p3 = opp_create_to_finish_metrics_p2.rename(columns = {'ttt':'event_team_name'})
    
    out_create_to_finish_metrics = pd.concat([teamed_create_to_finish_metrics,opp_create_to_finish_metrics_p3[teamed_create_to_finish_metrics.columns]], ignore_index = True)
    
    return out_create_to_finish_metrics
'''
def find_per_game_create_to_finish_metrics(data, col_name, team_name):

    new_df_f_out_teamed = data[(data[col_name] == team_name)]

    n_of_matches = len(new_df_f_out_teamed['game_id'].unique())
    create_to_finish_successful = new_df_f_out_teamed[(new_df_f_out_teamed['CREATE_FINISH'] != '') & (~new_df_f_out_teamed['CREATE_FINISH'].isnull())].groupby(by = [col_name,'game_id'])['game_phase_id'].count().reset_index().rename(columns = {'game_phase_id':'successful_create_to_finish'})
    create_to_finish_total = new_df_f_out_teamed.groupby(by = [col_name,'game_id'])['game_phase_id'].count().reset_index().rename(columns = {'game_phase_id':'total_create_to_finish'})
    create_to_finish_metrics = pd.merge(create_to_finish_total, create_to_finish_successful, on = [col_name,'game_id'])
    
    teamed_create_to_finish_metrics = create_to_finish_metrics[(create_to_finish_metrics[col_name] == team_name)]

    if col_name == 'against_team_name':
        add_prefix = 'Opponent '
    else:
        add_prefix = ''
        
    teamed_create_to_finish_metrics['Value'] = (teamed_create_to_finish_metrics['successful_create_to_finish']  / teamed_create_to_finish_metrics['total_create_to_finish']  * 100).round(2)
    
    teamed_create_to_finish_metrics['KPI'] = add_prefix+'Create to Finish Success Rate (%)'
    teamed_create_to_finish_metrics['n_of_matches'] = n_of_matches

    teamed_create_to_finish_metrics = teamed_create_to_finish_metrics.rename(columns = {col_name:'event_team_name'})
    
    return teamed_create_to_finish_metrics
    
def find_create_to_finish_metrics_by_tier(data, tier):

    new_df_f_out_teamed = data[(data['team_tier'] == tier)]
    new_df_f_out_opponents = data[(data['against_tier'] == tier)]

    n_of_matches = len(new_df_f_out_teamed['game_id'].unique())
    create_to_finish_successful = new_df_f_out_teamed[(new_df_f_out_teamed['CREATE_FINISH'] != '')].groupby(by = ['team_tier'])['game_phase_id'].count().reset_index().rename(columns = {'game_phase_id':'successful_create_to_finish'})
    create_to_finish_total = new_df_f_out_teamed.groupby(by = ['team_tier'])['game_phase_id'].count().reset_index().rename(columns = {'game_phase_id':'total_create_to_finish'})
    create_to_finish_metrics = pd.merge(create_to_finish_total, create_to_finish_successful, on = ['team_tier', 'game_id'])
    
    opp_n_of_matches = len(new_df_f_out_opponents['game_id'].unique())
    opp_create_to_finish_successful = new_df_f_out_opponents[(new_df_f_out_opponents['CREATE_FINISH'] != '')].groupby(by = ['against_tier'])['game_phase_id'].count().reset_index().rename(columns = {'game_phase_id':'opp_successful_create_to_finish'})
    opp_create_to_finish_total = new_df_f_out_opponents.groupby(by = ['against_tier'])['game_phase_id'].count().reset_index().rename(columns = {'game_phase_id':'opp_total_create_to_finish'})
    opp_create_to_finish_metrics = pd.merge(opp_create_to_finish_total, opp_create_to_finish_successful, on = ['against_tier'])
    
    teamed_create_to_finish_metrics = create_to_finish_metrics[(create_to_finish_metrics['event_team_name'] == tier)]
    opp_create_to_finish_metrics = opp_create_to_finish_metrics[(opp_create_to_finish_metrics['against_team_name'] == tier)]

    teamed_create_to_finish_metrics['Value'] = (teamed_create_to_finish_metrics['successful_create_to_finish']  / teamed_create_to_finish_metrics['total_create_to_finish']  * 100).round(2)
    opp_create_to_finish_metrics['Value'] = (opp_create_to_finish_metrics['opp_successful_create_to_finish']  / opp_create_to_finish_metrics['opp_total_create_to_finish']  * 100).round(2)
    
    teamed_create_to_finish_metrics['KPI'] = add_prefix+'Create to Finish Success Rate (%)'
    teamed_create_to_finish_metrics['n_of_matches'] = n_of_matches

    opp_create_to_finish_metrics['KPI'] = add_prefix+'Opponent Create to Finish Success Rate (%)'
    opp_create_to_finish_metrics['n_of_matches'] = opp_n_of_matches
    opp_create_to_finish_metrics = opp_create_to_finish_metrics.rename(columns = {'against_tier':'team_tier'})

    out_create_to_finish_metrics = pd.concat([teamed_create_to_finish_metrics,opp_create_to_finish_metrics[teamed_create_to_finish_metrics.columns]], ignore_index = True)
    
    return out_create_to_finish_metrics

def find_time_metrics_by_tier(data, tier):

    new_df_f_out_teamed = data[(data['team_tier'] == tier)]
    new_df_f_out_opponents = data[(data['against_tier'] == tier)]

    n_of_matches = len(new_df_f_out_teamed['game_id'].unique())
    in_possession_time_spent_per_team_phase = new_df_f_out_teamed.groupby(by = ['team_tier','against_tier', 'phase'])['time_diff'].sum().reset_index()
    out_of_possession_time_spent_per_team_phase_p1 = new_df_f_out_opponents.groupby(by = ['team_tier','against_tier', 'phase'])['time_diff'].sum().reset_index().rename(columns = {'against_tier':'TTT'})
    out_of_possession_time_spent_per_team_phase_p2 = out_of_possession_time_spent_per_team_phase_p1.rename(columns = {'team_tier':'against_tier'})
    out_of_possession_time_spent_per_team_phase = out_of_possession_time_spent_per_team_phase_p2.rename(columns = {'TTT':'team_tier'})
    
    out_of_possession_time_spent_per_team_phase['phase'] = out_of_possession_time_spent_per_team_phase['phase'].replace({
        'BUILD': '% IN HIGH BLOCK',
        'CREATE': '% IN MID BLOCK',
        'FINISH': '% IN LOW BLOCK',
        'TRANSITION D-A': '% IN TRANSITION A-D'
    })

    out_of_possession_time_spent_per_team_phase = out_of_possession_time_spent_per_team_phase[in_possession_time_spent_per_team_phase.columns]
    
    time_spent_metrics = pd.concat([in_possession_time_spent_per_team_phase, out_of_possession_time_spent_per_team_phase])
    
    time_spent_metrics['KPI'] = time_spent_metrics['phase'].replace({
        'BUILD': '% IN BUILD PHASE',
        'CREATE': '% IN CREATE PHASE',
        'FINISH': '% IN FINISH PHASE',
        'TRANSITION D-A': '% IN TRANSITION D-A'
    })
    
    time_spent_metrics['Value'] = (time_spent_metrics['time_diff'] / time_spent_metrics.groupby(by = ['against_tier','team_tier'])['time_diff'].transform('sum') * 100).round(2)
    
    teamed_time_metrics = time_spent_metrics[(time_spent_metrics['team_tier'] == tier)]
    teamed_time_metrics['n_of_matches'] = n_of_matches
    
    return teamed_time_metrics
    
def find_time_metrics(data, team_name):

    new_df_f_out_teamed = data[(data['event_team_name'] == team_name)]
    new_df_f_out_opponents = data[(data['against_team_name'] == team_name)]

    n_of_matches = len(new_df_f_out_teamed['game_id'].unique())
    in_possession_time_spent_per_team_phase = new_df_f_out_teamed.groupby(by = ['event_team_name', 'phase'])['time_diff'].sum().reset_index()
    out_of_possession_time_spent_per_team_phase = new_df_f_out_opponents.groupby(by = ['against_team_name', 'phase'])['time_diff'].sum().reset_index().rename(columns = {'against_team_name':'event_team_name','against_tier':'TTT'})
    
    out_of_possession_time_spent_per_team_phase['phase'] = out_of_possession_time_spent_per_team_phase['phase'].replace({
        'BUILD': '% IN HIGH BLOCK',
        'CREATE': '% IN MID BLOCK',
        'FINISH': '% IN LOW BLOCK',
        'TRANSITION D-A': '% IN TRANSITION A-D'
    })
    
    time_spent_metrics = pd.concat([in_possession_time_spent_per_team_phase, out_of_possession_time_spent_per_team_phase])
    
    time_spent_metrics['KPI'] = time_spent_metrics['phase'].replace({
        'BUILD': '% IN BUILD PHASE',
        'CREATE': '% IN CREATE PHASE',
        'FINISH': '% IN FINISH PHASE',
        'TRANSITION D-A': '% IN TRANSITION D-A'
    })
    
    time_spent_metrics['Value'] = (time_spent_metrics['time_diff'] / time_spent_metrics.groupby('event_team_name')['time_diff'].transform('sum') * 100).round(2)
    
    teamed_time_metrics = time_spent_metrics[(time_spent_metrics['event_team_name'] == team_name)]
    teamed_time_metrics['n_of_matches'] = n_of_matches
    
    return teamed_time_metrics

def find_create_to_finish_metrics(data, team_name):

    new_df_f_out_teamed = data[(data['event_team_name'] == team_name)]
    new_df_f_out_opponents = data[(data['against_team_name'] == team_name)]

    n_of_matches = len(new_df_f_out_teamed['game_id'].unique())
    in_possession_time_spent_per_team_phase = new_df_f_out_teamed.groupby(by = ['event_team_name', 'phase'])['time_diff'].sum().reset_index()
    out_of_possession_time_spent_per_team_phase = new_df_f_out_opponents.groupby(by = ['against_team_name', 'phase'])['time_diff'].sum().reset_index().rename(columns = {'against_team_name':'event_team_name'})
    

    n_of_matches = len(new_df_f_out_teamed['game_id'].unique())
    create_to_finish_successful = new_df_f_out_teamed[(new_df_f_out_teamed['CREATE_FINISH'] != '')].groupby(by = [col_name])['game_phase_id'].count().reset_index().rename(columns = {'game_phase_id':'successful_create_to_finish'})
    create_to_finish_total = new_df_f_out_teamed.groupby(by = [col_name])['game_phase_id'].count().reset_index().rename(columns = {'game_phase_id':'total_create_to_finish'})
    create_to_finish_metrics = pd.merge(create_to_finish_total, create_to_finish_successful, on = [col_name])
    
    teamed_create_to_finish_metrics = create_to_finish_metrics[(create_to_finish_metrics[col_name] == team_name)]

    if col_name == 'against_team_name':
        add_prefix = 'Opponent '
    else:
        add_prefix = ''
        
    teamed_create_to_finish_metrics['Value'] = ((teamed_create_to_finish_metrics['successful_create_to_finish']  / teamed_create_to_finish_metrics['total_create_to_finish'])  * 100).round(2)
    
    teamed_create_to_finish_metrics['KPI'] = add_prefix+'Create to Finish Success Rate (%)'
    teamed_create_to_finish_metrics['n_of_matches'] = n_of_matches

    teamed_create_to_finish_metrics = teamed_create_to_finish_metrics.rename(columns = {col_name:'event_team_name'})
    
    return teamed_create_to_finish_metrics
    
def safe_float_conversion(value):
        try:
            return float(value)
        except (ValueError, TypeError):
            return None
            
def new_description_of_create_phase(row):
    

    float_row_y_build_create = safe_float_conversion(row['end_y_BUILD_CREATE'])
    float_row_y_create_finish = safe_float_conversion(row['end_y_CREATE_FINISH'])
    float_row_y_create_build = safe_float_conversion(row['end_y_CREATE_BUILD'])
    float_row_y_finish_create = safe_float_conversion(row['end_y_FINISH_CREATE'])

    if (
            float(row['left']) > 0 and float(row['inside_left']) > 0 and
            (
                (float_row_y_create_finish is not None and float_row_y_create_finish > 80) or 
                (float_row_y_build_create is not None and float_row_y_build_create > 80) or 
                (float_row_y_create_build is not None and float_row_y_create_build > 80) or 
                (float_row_y_finish_create is not None and float_row_y_finish_create > 80)
            )
        ) or (
            float(row['left']) > 0 and 
            (
                (float_row_y_create_finish is not None and float_row_y_create_finish > 70) or 
                (float_row_y_build_create is not None and float_row_y_build_create > 70) or 
                (float_row_y_create_build is not None and float_row_y_create_build > 70) or 
                (float_row_y_finish_create is not None and float_row_y_finish_create > 70)
            )
        ):
            return 'Around'
    elif (
            float(row['right']) > 0 and float(row['inside_right']) > 0 and 
            (
                (float_row_y_create_finish is not None and float_row_y_create_finish < 20) or 
                (float_row_y_build_create is not None and float_row_y_build_create < 20) or 
                (float_row_y_create_build is not None and float_row_y_create_build < 20) or 
                (float_row_y_finish_create is not None and float_row_y_finish_create < 20)
            )
        ) or (
            float(row['right']) > 0 and 
            (
                (float_row_y_create_finish is not None and float_row_y_create_finish < 30) or 
                (float_row_y_build_create is not None and float_row_y_build_create < 30) or 
                (float_row_y_create_build is not None and float_row_y_create_build < 30) or 
                (float_row_y_finish_create is not None and float_row_y_finish_create < 30)
            )
        ):
            return 'Around'
    else:
        return row['phase_description']
   

In [237]:
new_df_f_out = pd.merge(concatenated_df_jjj, combined_df, on = ['game_id','create_phase_label','event_team_name'], how = 'outer')

new_df_f_out = new_df_f_out[new_df_f_out['team_phase_id'] != '']
new_df_f_out['event_outcome'] = new_df_f_out['event_outcome'].astype(float)

new_df_f_out.dtypes

game_id                       Int64
team_phase_id               float64
create_phase_label           object
vertical_pass_distance      float64
horizontal_pass_distance    float64
event_outcome               float64
n_of_events                 float64
half                        float64
start_time                  float64
end_time                    float64
left                        float64
inside_left                 float64
central                     float64
inside_right                float64
right                       float64
long_passes                 float64
switch_passes               float64
phase_description            object
event_team_name              object
against_team_name            object
phase                        object
time_diff                   float64
BUILD_CREATE                 object
BUILD_FINISH                 object
CREATE_BUILD                 object
CREATE_FINISH                object
FINISH_BUILD                 object
FINISH_CREATE               

In [238]:
new_df_f_out = new_df_f_out.sort_values(by = ['game_id','start_time'])#.fillna('')
new_df_f_out['unique_id'] = new_df_f_out['game_id'].astype(str)+'_'+new_df_f_out['event_team_name']+'_'+new_df_f_out['create_phase_label']

new_df_f_out['create_to_finish_phase_description'] = new_df_f_out.apply(lambda row: new_description_of_create_phase(row), axis=1)


In [239]:
display_all(new_df_f_out.groupby(by = ['phase']).count())

,game_id,team_phase_id,create_phase_label,vertical_pass_distance,horizontal_pass_distance,event_outcome,n_of_events,half,start_time,end_time,left,inside_left,central,inside_right,right,long_passes,switch_passes,phase_description,event_team_name,against_team_name,time_diff,BUILD_CREATE,BUILD_FINISH,CREATE_BUILD,CREATE_FINISH,FINISH_BUILD,FINISH_CREATE,event_x_BUILD_CREATE,event_x_BUILD_FINISH,event_x_CREATE_BUILD,event_x_CREATE_FINISH,event_x_FINISH_BUILD,event_x_FINISH_CREATE,event_y_BUILD_CREATE,event_y_BUILD_FINISH,event_y_CREATE_BUILD,event_y_CREATE_FINISH,event_y_FINISH_BUILD,event_y_FINISH_CREATE,end_x_BUILD_CREATE,end_x_BUILD_FINISH,end_x_CREATE_BUILD,end_x_CREATE_FINISH,end_x_FINISH_BUILD,end_x_FINISH_CREATE,end_y_BUILD_CREATE,end_y_BUILD_FINISH,end_y_CREATE_BUILD,end_y_CREATE_FINISH,end_y_FINISH_BUILD,end_y_FINISH_CREATE,unique_id,create_to_finish_phase_description
phase,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
BUILD,13170,13170,13170,13170,13170,13170,13170,13170,13170,13170,13170,13170,13170,13170,13170,13170,13170,13170,13170,13170,13170,4546,319,1623,0,72,0,4546,319,1623,0,72,0,4546,319,1623,0,72,0,4217,282,1606,0,66,0,4217,282,5695,5695,5695,5695,13170,13170
CREATE,19615,19615,19615,19615,19615,19615,19615,19615,19615,19615,19615,19615,19615,19615,19615,19615,19615,19615,19615,19615,19615,4555,0,1634,2782,0,758,4555,0,1634,2782,0,758,4555,0,1634,2782,0,758,4224,0,1615,2510,0,705,4224,0,8248,8248,8248,8248,19615,19615
FINISH,9772,9772,9772,9772,9772,9772,9772,9772,9772,9772,9772,9772,9772,9772,9772,9772,9772,9772,9772,9772,9772,0,318,0,2785,72,764,0,318,0,2785,72,764,0,318,0,2785,72,764,0,281,0,2514,65,712,0,281,3546,3546,3546,3546,9772,9772


In [240]:
new_df_f_out.columns

Index(['game_id', 'team_phase_id', 'create_phase_label',
       'vertical_pass_distance', 'horizontal_pass_distance', 'event_outcome',
       'n_of_events', 'half', 'start_time', 'end_time', 'left', 'inside_left',
       'central', 'inside_right', 'right', 'long_passes', 'switch_passes',
       'phase_description', 'event_team_name', 'against_team_name', 'phase',
       'time_diff', 'BUILD_CREATE', 'BUILD_FINISH', 'CREATE_BUILD',
       'CREATE_FINISH', 'FINISH_BUILD', 'FINISH_CREATE',
       'event_x_BUILD_CREATE', 'event_x_BUILD_FINISH', 'event_x_CREATE_BUILD',
       'event_x_CREATE_FINISH', 'event_x_FINISH_BUILD',
       'event_x_FINISH_CREATE', 'event_y_BUILD_CREATE', 'event_y_BUILD_FINISH',
       'event_y_CREATE_BUILD', 'event_y_CREATE_FINISH', 'event_y_FINISH_BUILD',
       'event_y_FINISH_CREATE', 'end_x_BUILD_CREATE', 'end_x_BUILD_FINISH',
       'end_x_CREATE_BUILD', 'end_x_CREATE_FINISH', 'end_x_FINISH_BUILD',
       'end_x_FINISH_CREATE', 'end_y_BUILD_CREATE', 'end_y_BUILD

In [241]:
new_df_f_out[new_df_f_out['game_id'].isin([2354433, 2354487, 2507730])].groupby(by = ['game_id']).count()

,team_phase_id,create_phase_label,vertical_pass_distance,horizontal_pass_distance,event_outcome,n_of_events,half,start_time,end_time,left,...,end_x_FINISH_BUILD,end_x_FINISH_CREATE,end_y_BUILD_CREATE,end_y_BUILD_FINISH,end_y_CREATE_BUILD,end_y_CREATE_FINISH,end_y_FINISH_BUILD,end_y_FINISH_CREATE,unique_id,create_to_finish_phase_description
game_id,,,,,,,,,,,,,,,,,,,,,


In [242]:
display_all(new_df_f_out[new_df_f_out['game_id'].isin([2180289])])

,game_id,team_phase_id,create_phase_label,vertical_pass_distance,horizontal_pass_distance,event_outcome,n_of_events,half,start_time,end_time,left,inside_left,central,inside_right,right,long_passes,switch_passes,phase_description,event_team_name,against_team_name,phase,time_diff,BUILD_CREATE,BUILD_FINISH,CREATE_BUILD,CREATE_FINISH,FINISH_BUILD,FINISH_CREATE,event_x_BUILD_CREATE,event_x_BUILD_FINISH,event_x_CREATE_BUILD,event_x_CREATE_FINISH,event_x_FINISH_BUILD,event_x_FINISH_CREATE,event_y_BUILD_CREATE,event_y_BUILD_FINISH,event_y_CREATE_BUILD,event_y_CREATE_FINISH,event_y_FINISH_BUILD,event_y_FINISH_CREATE,end_x_BUILD_CREATE,end_x_BUILD_FINISH,end_x_CREATE_BUILD,end_x_CREATE_FINISH,end_x_FINISH_BUILD,end_x_FINISH_CREATE,end_y_BUILD_CREATE,end_y_BUILD_FINISH,end_y_CREATE_BUILD,end_y_CREATE_FINISH,end_y_FINISH_BUILD,end_y_FINISH_CREATE,unique_id,create_to_finish_phase_description


In [243]:
new_df_f_out.groupby(by = ['event_team_name','against_team_name']).count()

game_id  team_phase_id  create_phase_label  \
event_team_name against_team_name                                               
Argentina U17   Belgium U17            101            101                 101   
                Brazil U17             111            111                 111   
                Fiji U17               136            136                 136   
                Germany U17            125            125                 125   
                Japan U17              111            111                 111   
...                                    ...            ...                 ...   
Wales U18       England U18            294            294                 294   
Zambia U17      Brazil U17              76             76                  76   
                Honduras U17           121            121                 121   
                Indonesia U17          106            106                 106   
                Mali U17               115            115                 115   

                                   vertical_pass_distance  \
event_team_name against_team_name                           
Argentina U17   Belgium U17                           101   
                Brazil U17                            111   
                Fiji U17                              136   
                Germany U17                           125   
                Japan U17                             111   
...                                                   ...   
Wales U18       England U18                           294   
Zambia U17      Brazil U17                             76   
                Honduras U17                          121   
                Indonesia U17                         106   
                Mali U17                              115   

                                   horizontal_pass_distance  event_outcome  \
event_team_name against_team_name                                            
Argentina U17   Belgium U17                             101            101   
                Brazil U17                              111            111   
                Fiji U17                                136            136   
                Germany U17                             125            125   
                Japan U17                               111            111   
...                                                     ...            ...   
Wales U18       England U18                             294            294   
Zambia U17      Brazil U17                               76             76   
                Honduras U17                            121            121   
                Indonesia U17                           106            106   
                Mali U17                                115            115   

                                   n_of_events  half  start_time  end_time  \
event_team_name against_team_name                                            
Argentina U17   Belgium U17                101   101         101       101   
                Brazil U17                 111   111         111       111   
                Fiji U17                   136   136         136       136   
                Germany U17                125   125         125       125   
                Japan U17                  111   111         111       111   
...                                        ...   ...         ...       ...   
Wales U18       England U18                294   294         294       294   
Zambia U17      Brazil U17                  76    76          76        76   
                Honduras U17               121   121         121       121   
                Indonesia U17              106   106         106       106   
                Mali U17                   115   115         115       115   

                                   ...  end_x_FINISH_BUILD  \
event_team_name against_team_name  ...                       
Argentina U17   Belgium U17      

In [244]:
new_df_f_out.event_team_name.unique()

array(['England U18', 'Wales U18', 'Norway U18', 'Russia U18',
       'Netherlands U18', 'Belgium U18', 'Portugal U18', 'Sweden U18',
       'Denmark U18', 'Austria U18', 'Croatia U18', 'Faroe Islands U18',
       'Switzerland U18', 'Australia U18', 'France U18', 'Japan U18',
       'Panama U17', 'Morocco U17', 'Indonesia U17', 'Ecuador U17',
       'Spain U17', 'Canada U17', 'Uzbekistan U17', 'Mali U17',
       'Brazil U17', 'IR Iran U17', 'New Caledonia U17', 'England U17',
       'Argentina U17', 'Senegal U17', 'France U17', 'Burkina Faso U17',
       'Korea Republic U17', 'USA U17', 'Mexico U17', 'Germany U17',
       'Venezuela U17', 'New Zealand U17', 'Japan U17', 'Poland U17',
       'Korea Republic U18', 'Morocco U18', 'Czechia U18', 'Germany U18',
       'Northern Ireland U18', 'Sweden U17', 'Ukraine U19', 'Poland U18',
       'Italy U17', 'Qatar U17', 'Bolivia U17', 'South Africa U17',
       'Portugal U17', 'Croatia U17', 'Costa Rica U17',
       'United Arab Emirates U17', 

In [245]:

opp_strengths_query = """
SELECT *
FROM `the-fa-performance-insights.staging_results_model.international_team_strengths`
"""

opp_strengths = pandas_gbq.read_gbq(opp_strengths_query, project_id = 'the-fa-analysis-and-insight')
opp_strengths_senior = opp_strengths[['country_name','senior_team_rank']].drop_duplicates()

opp_strengths_senior_1 = opp_strengths_senior[opp_strengths_senior['senior_team_rank'] <= 10]
opp_strengths_senior_2 = opp_strengths_senior[(opp_strengths_senior['senior_team_rank'] > 10) & (opp_strengths_senior['senior_team_rank'] <= 30) ]
opp_strengths_senior_3 = opp_strengths_senior[(opp_strengths_senior['senior_team_rank'] > 30) & (opp_strengths_senior['senior_team_rank'] <= 60) ]
opp_strengths_senior_4 = opp_strengths_senior[opp_strengths_senior['senior_team_rank'] > 60]

opp_strengths_senior_1['Tier'] = 1
opp_strengths_senior_2['Tier'] = 2
opp_strengths_senior_3['Tier'] = 3
opp_strengths_senior_4['Tier'] = 4

opp_strengths_senior = pd.concat([opp_strengths_senior_1,  opp_strengths_senior_2,  opp_strengths_senior_3,  opp_strengths_senior_4], ignore_index = True)
opp_strengths_senior['TeamType'] = "Men's Senior"
opp_strengths_senior = opp_strengths_senior.rename(columns = {'country_name':'TeamName'})

display_all(opp_strengths_senior.sort_values(by = ['senior_team_rank']))

Downloading: 100%|█████████████████████████████████████████████████████████████|


/var/folders/2m/31gtyphx2h3fz_nycnfgx60c0000gp/T/ipykernel_23352/1577128284.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  opp_strengths_senior_1['Tier'] = 1
/var/folders/2m/31gtyphx2h3fz_nycnfgx60c0000gp/T/ipykernel_23352/1577128284.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  opp_strengths_senior_2['Tier'] = 2
/var/folders/2m/31gtyphx2h3fz_nycnfgx60c0000gp/T/ipykernel_23352/1577128284.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using

,TeamName,senior_team_rank,Tier,TeamType
6,Germany,1,1,Men's Senior
2,England,2,1,Men's Senior
8,Spain,3,1,Men's Senior
5,France,4,1,Men's Senior
7,Italy,5,1,Men's Senior
1,Netherlands,6,1,Men's Senior
9,Portugal,7,1,Men's Senior
4,Brazil,8,1,Men's Senior
3,Belgium,9,1,Men's Senior
0,Argentina,10,1,Men's Senior


In [246]:
'''
opp_strengths_senior.to_gbq(
            destination_table='dev_vanni.senior_team_tier',
            project_id='the-fa-performance-insights',
            if_exists='replace'
        )
'''

"\nopp_strengths_senior.to_gbq(\n            destination_table='dev_vanni.senior_team_tier',\n            project_id='the-fa-performance-insights',\n            if_exists='replace'\n        )\n"

In [247]:
'''

opp_strengths_query = """
SELECT *
FROM `the-fa-analysis-and-insight.oppo_metrics.tier_rankings_table`
"""

opp_strengths = pandas_gbq.read_gbq(opp_strengths_query, project_id = 'the-fa-analysis-and-insight')
opp_strengths_senior = opp_strengths[opp_strengths['TeamType'] == "Men's Senior"]

'''

'\n\nopp_strengths_query = """\nSELECT *\nFROM `the-fa-analysis-and-insight.oppo_metrics.tier_rankings_table`\n"""\n\nopp_strengths = pandas_gbq.read_gbq(opp_strengths_query, project_id = \'the-fa-analysis-and-insight\')\nopp_strengths_senior = opp_strengths[opp_strengths[\'TeamType\'] == "Men\'s Senior"]\n\n'

In [248]:
display_all(opp_strengths_senior)

,TeamName,senior_team_rank,Tier,TeamType
0,Argentina,10,1,Men's Senior
1,Netherlands,6,1,Men's Senior
2,England,2,1,Men's Senior
3,Belgium,9,1,Men's Senior
4,Brazil,8,1,Men's Senior
5,France,4,1,Men's Senior
6,Germany,1,1,Men's Senior
7,Italy,5,1,Men's Senior
8,Spain,3,1,Men's Senior
9,Portugal,7,1,Men's Senior


In [249]:
opp_strengths_senior[opp_strengths_senior['TeamName'].str.contains('Canada')].head()

,TeamName,senior_team_rank,Tier,TeamType
49,Canada,47,3,Men's Senior


In [250]:
new_df_f_out_copy = new_df_f_out[(~new_df_f_out['event_team_name'].isnull()) & (~new_df_f_out['against_team_name'].isnull())]
new_df_f_out_copy['game_phase_id'] = new_df_f_out_copy['game_id'].astype(str) + '_' + new_df_f_out_copy['start_time'].astype(str) + '_' + new_df_f_out_copy['end_time'].astype(str)


/var/folders/2m/31gtyphx2h3fz_nycnfgx60c0000gp/T/ipykernel_23352/2150354210.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df_f_out_copy['game_phase_id'] = new_df_f_out_copy['game_id'].astype(str) + '_' + new_df_f_out_copy['start_time'].astype(str) + '_' + new_df_f_out_copy['end_time'].astype(str)


In [251]:
def find_opponent_strength(team_, opponent_):
        
        team_general = team_.replace(' U15', '').replace(' U16', '').replace(' U17', '').replace(' U18', '').replace(' U19', '').replace(' U20', '').replace(' U21', '').replace(' U22', '').replace(' U23', '').replace('Czech Republic','Czechia').replace('USA','United States').replace('IR Iran','Iran').replace('Bosnia and Herzegovina','Bosnia-Herzegovina').replace('DPR Korea','Korea DPR').replace('United Arab Emirates','UAE')
        opponent_general = opponent_.replace(' U15', '').replace(' U16', '').replace(' U17', '').replace(' U18', '').replace(' U19', '').replace(' U20', '').replace(' U21', '').replace(' U22', '').replace(' U23', '').replace('Czech Republic','Czechia').replace('USA','United States').replace('IR Iran','Iran').replace('Bosnia and Herzegovina','Bosnia-Herzegovina').replace('DPR Korea','Korea DPR').replace('United Arab Emirates','UAE')

        team_strengths_filt = opp_strengths_senior[opp_strengths_senior['TeamName'].str.contains(team_general)]
        opp_strengths_filt = opp_strengths_senior[opp_strengths_senior['TeamName'].str.contains(opponent_general)]

        if not team_strengths_filt.empty:
            team_out = team_strengths_filt['Tier'].iloc[0]
        else:
            team_out = None
            
        if not opp_strengths_filt.empty:
            opp_out = opp_strengths_filt['Tier'].iloc[0]
        else:
            opp_out = None

        return [team_out, opp_out]

new_df_f_out_copy['event_team_name'] = new_df_f_out_copy['event_team_name'].astype(str)
new_df_f_out_copy['against_team_name'] = new_df_f_out_copy['against_team_name'].astype(str)

new_df_f_out_copy[['team_tier', 'against_tier']] = ( new_df_f_out_copy.apply(lambda row: find_opponent_strength(row.event_team_name, row.against_team_name), axis=1) .apply(pd.Series) )
display_all(new_df_f_out_copy.head())

/var/folders/2m/31gtyphx2h3fz_nycnfgx60c0000gp/T/ipykernel_23352/2016987640.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df_f_out_copy['event_team_name'] = new_df_f_out_copy['event_team_name'].astype(str)
/var/folders/2m/31gtyphx2h3fz_nycnfgx60c0000gp/T/ipykernel_23352/2016987640.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df_f_out_copy['against_team_name'] = new_df_f_out_copy['against_team_name'].astype(str)
/var/folders/2m/31gtyphx2h3fz_nycnfgx60c0000gp/T/ipykernel_23352/201698764

,game_id,team_phase_id,create_phase_label,vertical_pass_distance,horizontal_pass_distance,event_outcome,n_of_events,half,start_time,end_time,left,inside_left,central,inside_right,right,long_passes,switch_passes,phase_description,event_team_name,against_team_name,phase,time_diff,BUILD_CREATE,BUILD_FINISH,CREATE_BUILD,CREATE_FINISH,FINISH_BUILD,FINISH_CREATE,event_x_BUILD_CREATE,event_x_BUILD_FINISH,event_x_CREATE_BUILD,event_x_CREATE_FINISH,event_x_FINISH_BUILD,event_x_FINISH_CREATE,event_y_BUILD_CREATE,event_y_BUILD_FINISH,event_y_CREATE_BUILD,event_y_CREATE_FINISH,event_y_FINISH_BUILD,event_y_FINISH_CREATE,end_x_BUILD_CREATE,end_x_BUILD_FINISH,end_x_CREATE_BUILD,end_x_CREATE_FINISH,end_x_FINISH_BUILD,end_x_FINISH_CREATE,end_y_BUILD_CREATE,end_y_BUILD_FINISH,end_y_CREATE_BUILD,end_y_CREATE_FINISH,end_y_FINISH_BUILD,end_y_FINISH_CREATE,unique_id,create_to_finish_phase_description,game_phase_id,team_tier,against_tier
79,2200777,9459.0,CREATE 1,36.120,31.076,2.0,3.0,1.0,0.538,9.079,1.0,0.0,1.0,0.0,0.0,1.0,0.0,Into / Onto,England U18,Wales U18,CREATE,8.541,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2200777_England U18_CREATE 1,Into / Onto,2200777_0.538_9.079,1,3
98,2200777,9459.0,CREATE 2,110.565,120.972,6.0,7.0,1.0,11.365,37.157,1.0,1.0,1.0,0.0,1.0,2.0,1.0,Into / Onto,England U18,Wales U18,CREATE,25.792,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2200777_England U18_CREATE 2,Into / Onto,2200777_11.365_37.157,1,3
1,2200777,18187.0,BUILD 1,0.000,0.000,1.0,1.0,1.0,61.245,65.859,0.0,1.0,0.0,0.0,0.0,0.0,0.0,Through,Wales U18,England U18,BUILD,4.614,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2200777_Wales U18_BUILD 1,Through,2200777_61.245_65.859,3,1
80,2200777,18187.0,CREATE 1,1.785,0.408,0.0,1.0,1.0,65.859,70.148,0.0,1.0,0.0,0.0,0.0,0.0,0.0,Through,Wales U18,England U18,CREATE,4.289,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2200777_Wales U18_CREATE 1,Through,2200777_65.859_70.148,3,1
113,2200777,9459.0,CREATE 3,64.575,86.496,6.0,6.0,1.0,70.148,88.941,1.0,1.0,1.0,0.0,1.0,1.0,1.0,Into / Onto,England U18,Wales U18,CREATE,18.793,NaN,NaN,NaN,Long,NaN,NaN,NaN,NaN,NaN,56.5,NaN,NaN,NaN,NaN,NaN,47.3,NaN,NaN,NaN,NaN,NaN,86.1,NaN,NaN,NaN,NaN,,74.4,,,2200777_England U18_CREATE 3,Around,2200777_70.148_88.941,1,3


In [252]:
new_df_f_out_copy[(new_df_f_out_copy['team_tier'].isnull()) | (new_df_f_out_copy['against_tier'].isnull())][['event_team_name','against_team_name','team_tier','against_tier']].drop_duplicates()


,event_team_name,against_team_name,team_tier,against_tier


In [253]:
new_df_f_out_copy = new_df_f_out_copy[(~new_df_f_out_copy['team_tier'].isnull()) & (~new_df_f_out_copy['against_tier'].isnull())]
len(new_df_f_out_copy['game_id'].unique())

209

In [254]:
new_df_f_out_copy[(new_df_f_out_copy['event_team_name'] == 'England U21') & (new_df_f_out_copy['against_team_name'] == 'Andorra U21')]

,game_id,team_phase_id,create_phase_label,vertical_pass_distance,horizontal_pass_distance,event_outcome,n_of_events,half,start_time,end_time,...,end_y_BUILD_FINISH,end_y_CREATE_BUILD,end_y_CREATE_FINISH,end_y_FINISH_BUILD,end_y_FINISH_CREATE,unique_id,create_to_finish_phase_description,game_phase_id,team_tier,against_tier


In [255]:
display_all(new_df_f_out_copy[['game_id','team_tier','against_tier']].drop_duplicates().groupby(by = ['team_tier','against_tier'])['game_id'].count())

team_tier  against_tier
1          1               30
           2               37
           3               24
           4               32
2          1               37
           2                5
           3               15
           4               20
3          1               24
           2               15
           3                7
           4               21
4          1               32
           2               20
           3               21
           4               18
Name: game_id, dtype: Int64

In [256]:
new_df_f_out_copy[new_df_f_out_copy['event_team_name'] == 'Romania U21']

,game_id,team_phase_id,create_phase_label,vertical_pass_distance,horizontal_pass_distance,event_outcome,n_of_events,half,start_time,end_time,...,end_y_BUILD_FINISH,end_y_CREATE_BUILD,end_y_CREATE_FINISH,end_y_FINISH_BUILD,end_y_FINISH_CREATE,unique_id,create_to_finish_phase_description,game_phase_id,team_tier,against_tier


In [257]:
team_ = 'Germany U18'
print(len(new_df_f_out_copy[(new_df_f_out_copy['event_team_name'] ==  team_) & (new_df_f_out_copy['CREATE_FINISH'].isnull()) & (new_df_f_out_copy['CREATE_FINISH'] != '')]))
print(len(new_df_f_out_copy[(new_df_f_out_copy['event_team_name'] ==  team_) & (new_df_f_out_copy['phase'] == 'CREATE')]))

#display_all(new_df_f_out_copy[new_df_f_out_copy['event_team_name'] == 'Germany U21'].head())

285
127


In [258]:
display_all(new_df_f_out_copy[(~new_df_f_out_copy['BUILD_FINISH'].isnull()) | (~new_df_f_out_copy['CREATE_FINISH'].isnull())].head())

,game_id,team_phase_id,create_phase_label,vertical_pass_distance,horizontal_pass_distance,event_outcome,n_of_events,half,start_time,end_time,left,inside_left,central,inside_right,right,long_passes,switch_passes,phase_description,event_team_name,against_team_name,phase,time_diff,BUILD_CREATE,BUILD_FINISH,CREATE_BUILD,CREATE_FINISH,FINISH_BUILD,FINISH_CREATE,event_x_BUILD_CREATE,event_x_BUILD_FINISH,event_x_CREATE_BUILD,event_x_CREATE_FINISH,event_x_FINISH_BUILD,event_x_FINISH_CREATE,event_y_BUILD_CREATE,event_y_BUILD_FINISH,event_y_CREATE_BUILD,event_y_CREATE_FINISH,event_y_FINISH_BUILD,event_y_FINISH_CREATE,end_x_BUILD_CREATE,end_x_BUILD_FINISH,end_x_CREATE_BUILD,end_x_CREATE_FINISH,end_x_FINISH_BUILD,end_x_FINISH_CREATE,end_y_BUILD_CREATE,end_y_BUILD_FINISH,end_y_CREATE_BUILD,end_y_CREATE_FINISH,end_y_FINISH_BUILD,end_y_FINISH_CREATE,unique_id,create_to_finish_phase_description,game_phase_id,team_tier,against_tier
113,2200777,9459.0,CREATE 3,64.575,86.496,6.0,6.0,1.0,70.148,88.941,1.0,1.0,1.0,0.0,1.0,1.0,1.0,Into / Onto,England U18,Wales U18,CREATE,18.793,NaN,NaN,NaN,Long,NaN,NaN,NaN,NaN,NaN,56.5,NaN,NaN,NaN,NaN,NaN,47.3,NaN,NaN,NaN,NaN,NaN,86.1,NaN,NaN,NaN,NaN,,74.4,,,2200777_England U18_CREATE 3,Around,2200777_70.148_88.941,1,3
182,2200777,9459.0,FINISH 1,58.590,50.864,6.0,6.0,1.0,88.941,101.493,0.0,0.0,1.0,2.0,1.0,0.0,0.0,Through,England U18,Wales U18,FINISH,12.552,NaN,NaN,NaN,Long,NaN,Carry - Through Inside,NaN,NaN,NaN,56.5,NaN,46.6,NaN,NaN,NaN,47.3,NaN,36.4,NaN,NaN,NaN,86.1,NaN,47.4,NaN,NaN,,74.4,,40.4,2200777_England U18_FINISH 1,Through,2200777_88.941_101.493,1,3
84,2200777,18187.0,CREATE 11,28.140,31.008,3.0,3.0,1.0,990.596,999.135,0.0,0.0,1.0,1.0,0.0,0.0,0.0,Through,Wales U18,England U18,CREATE,8.539,NaN,NaN,NaN,Error,NaN,NaN,NaN,NaN,NaN,50.6,NaN,NaN,NaN,NaN,NaN,38.4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,,,,2200777_Wales U18_CREATE 11,Through,2200777_990.596_999.135,3,1
209,2200777,18187.0,FINISH 6,0.000,0.000,1.0,1.0,1.0,1029.276,1038.755,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Through,Wales U18,England U18,FINISH,9.479,NaN,NaN,NaN,Error,NaN,NaN,NaN,NaN,NaN,50.6,NaN,NaN,NaN,NaN,NaN,38.4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,,,,2200777_Wales U18_FINISH 6,Through,2200777_1029.276_1038.755,3,1
106,2200777,9459.0,CREATE 26,54.495,24.276,4.0,5.0,1.0,1604.520,1628.680,1.0,1.0,0.0,0.0,0.0,0.0,0.0,Through,England U18,Wales U18,CREATE,24.160,NaN,NaN,NaN,Carry - Around,NaN,NaN,NaN,NaN,NaN,88.9,NaN,NaN,NaN,NaN,NaN,79.4,NaN,NaN,NaN,NaN,NaN,89.7,NaN,NaN,NaN,NaN,,88.3,,,2200777_England U18_CREATE 26,Around,2200777_1604.52_1628.68,1,3


In [259]:
new_df_f_out_copy[(new_df_f_out_copy['event_team_name'] ==  team_) & (new_df_f_out_copy['create_to_finish_phase_description'] != '') & (~new_df_f_out_copy['create_to_finish_phase_description'].isnull())]['game_phase_id'].count()
new_df_f_out_copy[(new_df_f_out_copy['event_team_name'] ==  team_)]['game_phase_id'].count()

print(new_df_f_out_copy[(new_df_f_out_copy['event_team_name'] ==  team_)].groupby(by = ['event_team_name','against_team_name','game_id'])['game_phase_id'].count().reset_index().rename(columns = {'game_phase_id':'total_create_to_finish'}))
print(new_df_f_out_copy[(new_df_f_out_copy['event_team_name'] ==  team_) & (new_df_f_out_copy['create_to_finish_phase_description'] != '') & (~new_df_f_out_copy['create_to_finish_phase_description'].isnull())].groupby(by = ['event_team_name','against_team_name','game_id'])['game_phase_id'].count().reset_index().rename(columns = {'game_phase_id':'successful_create_to_finish'}))

  event_team_name against_team_name  game_id  total_create_to_finish
0     Germany U18       England U18  2438169                      96
1     Germany U18       England U18  2490778                     122
2     Germany U18   Netherlands U18  2607980                      79
  event_team_name against_team_name  game_id  successful_create_to_finish
0     Germany U18       England U18  2438169                           96
1     Germany U18       England U18  2490778                          122
2     Germany U18   Netherlands U18  2607980                           79


In [260]:
display_all(new_df_f_out_copy[new_df_f_out_copy['time_diff'] == ''])

,game_id,team_phase_id,create_phase_label,vertical_pass_distance,horizontal_pass_distance,event_outcome,n_of_events,half,start_time,end_time,left,inside_left,central,inside_right,right,long_passes,switch_passes,phase_description,event_team_name,against_team_name,phase,time_diff,BUILD_CREATE,BUILD_FINISH,CREATE_BUILD,CREATE_FINISH,FINISH_BUILD,FINISH_CREATE,event_x_BUILD_CREATE,event_x_BUILD_FINISH,event_x_CREATE_BUILD,event_x_CREATE_FINISH,event_x_FINISH_BUILD,event_x_FINISH_CREATE,event_y_BUILD_CREATE,event_y_BUILD_FINISH,event_y_CREATE_BUILD,event_y_CREATE_FINISH,event_y_FINISH_BUILD,event_y_FINISH_CREATE,end_x_BUILD_CREATE,end_x_BUILD_FINISH,end_x_CREATE_BUILD,end_x_CREATE_FINISH,end_x_FINISH_BUILD,end_x_FINISH_CREATE,end_y_BUILD_CREATE,end_y_BUILD_FINISH,end_y_CREATE_BUILD,end_y_CREATE_FINISH,end_y_FINISH_BUILD,end_y_FINISH_CREATE,unique_id,create_to_finish_phase_description,game_phase_id,team_tier,against_tier


In [261]:
single_games_team_metrics = pd.DataFrame()

for team_name in new_df_f_out_copy['event_team_name'].unique():

    print(team_name)
    
    game_teamed_time_metrics = find_per_game_time_metrics(new_df_f_out_copy, team_name)
    print(len(game_teamed_time_metrics['game_id'].unique()))
    game_teamed_create_finish_metrics = find_per_game_create_to_finish_metrics(new_df_f_out_copy, 'event_team_name', team_name)
    opp_game_teamed_create_finish_metrics = find_per_game_create_to_finish_metrics(new_df_f_out_copy, 'against_team_name', team_name)
    
    single_games_team_metrics = pd.concat([single_games_team_metrics, game_teamed_time_metrics, game_teamed_create_finish_metrics], ignore_index = True)

display_all(single_games_team_metrics.sort_values(by = ['game_id','event_team_name']).head(20))

England U18
48
Wales U18
3
Norway U18
2
Russia U18
1
Netherlands U18
7
Belgium U18
4
Portugal U18
5
Sweden U18
3
Denmark U18
1
Austria U18
1
Croatia U18
2
Faroe Islands U18
1
Switzerland U18
3
Australia U18
1
France U18
5
Japan U18
1
Panama U17
6
Morocco U17
11
Indonesia U17
6
Ecuador U17
4
Spain U17
5
Canada U17
7
Uzbekistan U17
10
Mali U17
12
Brazil U17
13
IR Iran U17
4
New Caledonia U17
6
England U17
9
Argentina U17
11
Senegal U17
8
France U17
12
Burkina Faso U17
9
Korea Republic U17
7
USA U17
8
Mexico U17
9
Germany U17
11
Venezuela U17
8
New Zealand U17
6
Japan U17
9
Poland U17
2
Korea Republic U18
1
Morocco U18
4
Czechia U18
2
Germany U18
3
Northern Ireland U18
1
Sweden U17
1
Ukraine U19
1
Poland U18
2
Italy U17
8
Qatar U17
3
Bolivia U17
3
South Africa U17
4
Portugal U17
8
Croatia U17
4
Costa Rica U17
3
United Arab Emirates U17
3
Belgium U17
4
Fiji U17
3
Tunisia U17
4
Egypt U17
4
Haiti U17
3
Switzerland U17
6
Côte d'Ivoire U17
3
Colombia U17
4
El Salvador U17
3
DPR Korea U17
5
Hon

,event_team_name,game_id,against_team_name,team_tier,against_tier,phase,time_diff,KPI,Value,n_of_matches,total_create_to_finish,successful_create_to_finish
0,England U18,2200777,Wales U18,1.0,3.0,BUILD,614.164,% IN BUILD PHASE,20.12,48,NaN,NaN
1,England U18,2200777,Wales U18,1.0,3.0,CREATE,1017.954,% IN CREATE PHASE,33.35,48,NaN,NaN
2,England U18,2200777,Wales U18,1.0,3.0,FINISH,430.967,% IN FINISH PHASE,14.12,48,NaN,NaN
147,England U18,2200777,Wales U18,1.0,3.0,% IN HIGH BLOCK,338.170,% IN HIGH BLOCK,11.08,48,NaN,NaN
148,England U18,2200777,Wales U18,1.0,3.0,% IN MID BLOCK,469.936,% IN MID BLOCK,15.39,48,NaN,NaN
149,England U18,2200777,Wales U18,1.0,3.0,% IN LOW BLOCK,181.368,% IN LOW BLOCK,5.94,48,NaN,NaN
291,England U18,2200777,NaN,NaN,NaN,NaN,NaN,Create to Finish Success Rate (%),10.69,48,131.0,14.0
339,Wales U18,2200777,England U18,3.0,1.0,BUILD,338.170,% IN BUILD PHASE,11.08,3,NaN,NaN
340,Wales U18,2200777,England U18,3.0,1.0,CREATE,469.936,% IN CREATE PHASE,15.39,3,NaN,NaN
341,Wales U18,2200777,England U18,3.0,1.0,FINISH,181.368,% IN FINISH PHASE,5.94,3,NaN,NaN


In [264]:
display_all(single_games_team_metrics[['event_team_name','game_id']].groupby(by = ['event_team_name'])['game_id'].count())

event_team_name
Argentina U17                  77
Australia U18                   7
Austria U17                    56
Austria U18                     7
Belgium U17                    28
Belgium U18                    28
Bolivia U17                    20
Bosnia and Herzegovina U18      7
Brazil U17                     91
Burkina Faso U17               62
Canada U17                     49
Canada U18                      7
Chile U17                      20
Colombia U17                   28
Costa Rica U17                 19
Croatia U17                    28
Croatia U18                    14
Czechia U17                    28
Czechia U18                    14
Côte d'Ivoire U17              21
DPR Korea U17                  35
Denmark U18                     7
Ecuador U17                    28
Egypt U17                      28
El Salvador U17                21
England U17                    63
England U18                   339
Faroe Islands U18               7
Fiji U17                       2

In [263]:
single_games_team_metrics.to_gbq(
            destination_table='dev_vanni.benchmark_phases_by_games',
            project_id='the-fa-performance-insights',
            if_exists=replaceOrAppend
        )

/var/folders/2m/31gtyphx2h3fz_nycnfgx60c0000gp/T/ipykernel_23352/3236815090.py:1: FutureWarning: to_gbq is deprecated and will be removed in a future version. Please use pandas_gbq.to_gbq instead: https://pandas-gbq.readthedocs.io/en/latest/api.html#pandas_gbq.to_gbq
  single_games_team_metrics.to_gbq(
100%|██████████████████████████████████████████| 1/1 [00:00<00:00, 11275.01it/s]
